<a id="top"></a>

# Microlensing Tools
<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;">

<a href="https://colab.research.google.com/github/rges-pit/data-challenge-notebooks/blob/main/Extras/Microlensing_Tools.ipynb" target="_blank" rel="noopener"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you would like an introduction to python notebooks, please read this tutorial: https://medium.com/codingthesmartway-com-blog/getting-started-with-jupyter-notebook-for-python-4e7082bd5d46

## Learning Goals

The goal of this notebook is to demonstrate how to use various event fitting tools to analyze the Roman simulations.This notebook is intended as a single-source reference for how to use the tools, and is not intended to be a comprehensive analysis of the Roman simulations.

## Environment Set-Up

**Dependencies**

This notebook has the following base dependencies:

  - `python` (v>=3.11 recommended)
  - `numpy`
  - `matplotlib`
  - `pandas`
  - `scipy`
  - `jupyter`
  - `ipython`
  - `astropy`
  - `emcee` 
  - `corner`
  - `popclass`
  - `tqdm` (remaining packages are not currently available on conda)
  - `pathos`
  - `VBMicrolensing`
  - `MulensModel` (v=3.4.0)
  - `pyLIMA`
  - `RTModel`

<!-- NEXUS-ONLY -->
> ### Running on the Roman Research Nexus
> If you are following this notebook on the Roman Research Nexus (RRN) these packages are preinstalled on the RGES-PIT's kernel:
> 1. In notebooks, open **Kernel > Change Kernel** menu and select `RGES PIT Nexus` or select the kernel via the Kernel menu or clicking on the \"kernel status display\" to the top right with the activity circle.
> 1. In a terminal, if needed,  run `mamba activate rges-pit-dc` to select the correct mamba environment."
> 1. Store any files you generate in your home directory or team space; the `/roman/nexus-notebooks/notebooks` directory is read-only.

You can install these packages by running the next cell

In [ ]:
# COLAB etc
%pip install numpy matplotlib pandas scipy ipython astropy pathos emcee corner tqdm VBMicrolensing MulensModel==3.4.0 pyLIMA RTModel popclass

### Imports

In [ ]:
#@title General Imports

# system tools
import os
import time
import shutil
import subprocess
from pathlib import Path
import tempfile
import zipfile
import warnings
# Mute only this ArviZ FutureWarning message 
# We have pinned corner and emcee versions to avoid this becoming an issue
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    module=r"arviz(\.|$)",
    message=r".*ArviZ is undergoing a major refactor.*",
)
# Hide noisy ERFA warnings caused by trial steps far from valid epochs
try:
    from erfa import ErfaWarning
except Exception:
    ErfaWarning = Warning

# data analysis tools
import numpy as np
import matplotlib.pyplot as plt
from IPython import get_ipython
from IPython.display import display
import pandas as pd
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from scipy.optimize import approx_fprime
import emcee
import corner
import astropy.units as u
from astropy.coordinates import SkyCoord
try:
    from google.colab import sheets  # this will only work if you are running on Colab
except:
    pass

# web scrapping tools
import requests

# parallel processing tools
from pathos.multiprocessing import ProcessingPool as Pool  # for multiprocessing inside jupyter
import multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor as ThreadPool

/opt/anaconda3/envs/rges-pit-dc/lib/python3.11/site-packages/arviz/__init__.py:39: FutureWarning: 
ArviZ is undergoing a major refactor to improve flexibility and extensibility while maintaining a user-friendly interface.
Some upcoming changes may be backward incompatible.
For details and migration guidance, visit: https://python.arviz.org/en/latest/user_guide/migration_guide.html
  warn(


## Introduction

Welcome to the **Roman Microlensing Data Challenge 2026 (RMDC26)** notebook on **Microlensing Tools**. If you have not already, you can click the link below to access the first notebook in this series: 

[**RMDC26/Nexus Workflow**](a_rmdc26_workflow.ipynb)

The data challenge is intended to be a semi-realistic representation of the data volume and type expected from the Roman Galactic Bulge Time Domain Survey. Specifically for microlensing events and microlensing false positives.

This notebook is brought to you by the RGES-PIT and is inteneded to be an introductory workbook for users new to microlensing event fitting and physical parameter estimation, users who would like a refresher, or users who would like a comprehensive introduction to tools they have not used before. It is not recommended that you complete this notebook in its entirety. You should make use of the tool look-up table and index to navigate to the section relevant to your interest or issue.

Our aim is to provide you with a centralized resource covering open-source microlensing code applications. It is not designed to provide you with a realistic view of what working with bulk microlensing data involves or the general procedures undertaken to demistify microlensings event data. We will link to resources to understand procedures that are currently implemented in microlensing detection and analysis.  

### What data are used in this notebook?

This notebook primarily uses lightcurves from the [2018 WFIRST Data Challenge](https://www.microlensing-source.org/data-challenge/). The data will be either fetched from an s3 bucket or [cloned](https://docs.github.com/en/repositories/creating-and-managing-repositories/cloning-a-repository) and sorted in later sections, depending on your environment. Because the 2018 WFIRST Data Challenge concluded in 2018, we can make full use of the [parameter "truths"](https://en.wikipedia.org/wiki/Statistical_parameter#:~:text=parameter%20describes%20the-,true%20value,-calculated%20from%20the) for each event to verify our fitting processes.

The dataset consists of two lightcurve files for each event or star, representing the data from Roman's `W149` and `Z087` filters. The files are in ASCII format with the columns BJD, Aperture_Magnitude and Error, and follow the file-naming convention: `ulwdc1_nnn_[W149/Z087].txt`

Supplementary files were also provided including `wfirst_ephemeris.txt`, which contains the `BJD` and 3D spacecraft location within the solar system. Information was provided on the surface-brightness color relation for `Z087-W149` to enable lens masses to be determined where applicable.

It should be noted that in the simulated data, the inertial frame of reference was defined with the $x$-axis increasing from the binary primary lens object towards the secondary (less massive) lens object at `t0`, the time of closest approach to the primary lens. If viewed from the solar system barycenter, the inertial frame moves at the relative velocity `vlens_CoM - vobserver(t0)`. The inclination of the orbit is a counter-clockwise rotation about the $x$-axis. $\alpha$ is the angle that the source trajectory made with the $x$-axis (if parallax was 0). Where finite source effects were significant, a linear limb darkening law was applied.

<!--
### <font face="Helvetica" size="5"> What lightcurve models are included in this notebook? </font>

The first data challenge contained the following kinds of lightcurves:

* Cataclysmic Variable Star (CVS) false positive
* Single lens microlensing events
* Binary lens microlensing events

You can expect a much broader set in the current data challenge with the addition of binary source microlensing events and more higher-order effects, including parallax, lens orbital motion, . Higher-order effects included in the first data challenge, which are still included in this data challenge, are finite source effects and ... .

Additional data types included in Data challenge 2 include:
* Astrometric timeseries
* Lens flux measurement
-->

### <font face="Helvetica" size="5"> What codes are used in this notebook? </font>

Some open source microlensing codes:
<!-- add a column for pip and conda -->
| Name | Notes | Maintained | Link | Covered here |
| :-: | :-: | :-: | :-: | :-: |
| MuLensModel | User friendly, single- and binary-lens fitting code. | Yes ([Poleski](https://github.com/rpoleski)) | [GitHub](https://github.com/rpoleski/MulensModel) | [Yes](#mulensmodel) |
| BAGEL | Incorporates photometric and astrometric microlensing. | Yes ([Moving Universe Lab](https://github.com/MovingUniverseLab)) | [GitHub](https://github.com/MovingUniverseLab/BAGLE_Microlensing) | [No](#bagel) |
| VBMicroelensing | A more general version of the binary-lens code [VBBL](https://github.com/valboz/VBBinaryLensing). VBMicrolensing <br>is a tool for efficient computation in gravitational microlensing events <br>using the advanced contour integration method, supporting single, binary <br>and multiple lenses. | Yes ([Bozza](https://github.com/valboz)) | [GitHub](https://github.com/valboz/VBMicrolensing) | [No](#vbmicrolensing)
| pyLIMA | pyLIMA is the first open source software for modeling microlensing <br>events. It should be flexible enough to handle your data and fit it. You can <br>also practice by simulating events. Useful for space-based observations. | Yes ([Bachelet](https://github.com/ebachelet)) | [GitHub](https://github.com/ebachelet/pyLIMA) | [Yes](#pylima) |
| RTModel | Hands-off model fitting with built in model <i>"interpretation"</i> <br>(e.g. determing single-lens vs binary-lens arrangement) | Yes ([Bozza](https://github.com/valboz)) | [Github](https://github.com/valboz/RTModel) | [Yes](RTModel) |
| eesunhong | No general description found. <br>See [Bennett and Rhie (1996)](https://ui.adsabs.harvard.edu/abs/1996ApJ...472..660B/abstract) and [Bennett (2010)](https://ui.adsabs.harvard.edu/abs/2010ApJ...716.1408B/abstract) | Yes ([Bennett]()) | [GitHub](https://github.com/golmschenk/eesunhong) | No |
| pyLIMASS | Addition to pyLIMA for estimating physical properties of the lens <br>system. See [Bachelet, Hundertmark, and Calchi Novati (2024)](https://ui.adsabs.harvard.edu/abs/2024AJ....168...24B/abstract) | Yes ([Bachelet](https://github.com/ebachelet)) | [GitHub](https://github.com/ebachelet/pyLIMA/tree/master/pyLIMA/pyLIMASS) | [Yes](#pylima) |
| popclass | Provides a flexible, probabilistic framework for classifying the lens <br>of a gravitational microlensing event. | Yes ([LLNL](https://github.com/LLNL)) | [GitHub](https://github.com/LLNL/popclass) | [Yes](#popclass) |
| muLAn | Designed for fitting Roman microlensing lightcurve data. | No ([Cassan](https://github.com/ArnaudCassan)/[Ranc](https://github.com/clementranc)) | [GitHub](https://github.com/muLAn-project/muLAn) | [No](#muLAN) |
| triplelens | Calculates light curves and image positions for triple microlensing <br>systems. (When the mass ratio is small (below ~ 1e-5), the solutions <br>from the lens equation solver are more accurate when the origin <br>of the coordinate system is set to be close to the smallest mass.) | No ([Kuang](https://github.com/rkkuang)) | [GitHub](https://github.com/rkkuang/triplelens) | No |
| SingleLensFitter | Fits single lens events with finite source effects | No ([Albrow](https://github.com/MichaelDAlbrow)) | [GitHub](https://github.com/MichaelDAlbrow/SingleLensFitter) | No |
<br>

> This is a replica of the tools list on the [RGES-PIT website](https://rges-pit.org/tools). We would like this table to be a comprehensive list, so if you have an open source code you would like to contribute, please refer to the [contributing documentation](https://github.com/rges-pit/data-challenge-notebooks/blob/main/CONTRIBUTING.md), open a discussion on the [GitHub repo](https://github.com/rges-pit/data-challenge-notebooks/discussions), or submit to or website [portal](https://rges-pit.org/submit/).

### Microlensing learning resources

* [RGES-PIT Minicourse](https://rges-pit.org/outreach_mini_landing/)

  The RGES PIT has developed a microlensing mini course for select students to participate in various Roman-related lectures and activities during the Summer of 2025. The virtual lectures were held in mid May 2025; you can find lecture materials, recordings, and assignments.

* [Microlensing Source](https://www.microlensing-source.org/)

  Microlensing Source is a resource center for all aspects of gravitational microlensing. It aims to make microlensing more accessible for anyone with an interest in the subject - including students considering a career in the field, citizen scientists and those looking for a ready reference.

* [The Microlenser's Guide to the Galaxy](https://amberlee2427.github.io/TheMicrolensersGuideToTheGalaxy/)

  The goal of this project is to create an all-encompassing collection of Jupyter notebooks—your trusty companions for engaging exercises related to microlensing. Through these notebooks, the insights and experiences of microlensing veterins can light your path as you embark on your journey of discovery and exploration through scientific research.

* [2017 Sagan Workshop](http://nexsci.caltech.edu/workshop/2017/)

  The 2017 Sagan Summer Workshop focus on searching for planets with Roman (previously known as WFIRST) microlensing. Leaders in the field will discussed the importance of microlensing to understanding planetary populations and demographics, especially beyond the snow line. They reviewed the microlensing method, both in the context of current capabilities and the future Roman microlensing survey. In addition, speakers addressed the broad potential of the Romans's Wide Field Imaging microlensing survey for (non-microlensing) science in the galactic bulge. Attendees participated in hands-on group projects related to the Roman microlensing planet survey and had the opportunity to present their own work through short presentations (research POPs) and posters.
  The recordings from this workshop can be found [here](https://www.youtube.com/watch?v=QPfKucBb9B8&list=PLIbTYGsIVYthWRS14eCEK8SK9IOTcaYsf)

* [Glossary of Terms](https://www.microlensing-source.org/glossary/)

  This glossary, from Microlensing Source, is intended as a quick reference, particularly to disambiguate the different symbol sets used by different authors over time. Interested readers are referred to the references at the bottom for a full discussion, especially Skowron et al. (2011), and to the Learning Resources menu.

> This is also a replica of the background material listed on the [resources](https://rges-pit.org/resources/#:~:text=Background%20material,Jupyter%20Notebook%20Course%20%C2%A0) page of the RGES-PIT website.

## Collecting the Data

In this section and we will clone our data from GitHub and organize it in to sensible dataframes.

In [ ]:
#@title Cloning the GitHub repository
if not os.path.exists("data-challenge-1"):

    # Clone
    !printf "🐑 " && git clone --depth 1 https://github.com/microlensing-data-challenge/data-challenge-1.git
else:
    print("ℹ️ data-challenge-1 already exists; skipping clone")

if not os.path.exists("data-challenge-1/lc"):
    # Extract with lightweight progress dots every N checkpoints
    print("\n🗃️ Extracting files")
    # extract with progress dots, if supported
    try:
        subprocess.run(["tar", "-xzf", "data-challenge-1/lc.tar.gz", "-C",
                        "data-challenge-1/", "--checkpoint=1000",
                        "--checkpoint-action=dot"], check=True)
    except:
        subprocess.run(["tar", "-xzf", "data-challenge-1/lc.tar.gz",
                        "-C", "data-challenge-1/"], check=True)

print("\n✅ Lightcurve files are ready.")

ℹ️ data-challenge-1 already exists; skipping clone

🗃️ Extracting files


tar: Option --checkpoint=1000 is not supported
Usage:
  List:    tar -tf <archive-filename>
  Extract: tar -xf <archive-filename>
  Create:  tar -cf <archive-filename> [filenames...]
  Help:    tar --help




✅ Lightcurve files are ready.


### Single Lens Events

This dataset includes 293 lightcurve, 74 of which are single lens events. We can cheat a little and specifically pull out the events that we know to be single lenses, keeping the challenge tractable for completion within the hour, with the added benefit of making the strangley organized `master_file.txt` easier to wrangle.

In [19]:
#@title Putting everything in a tidy data frame
master_file = './data-challenge-1/Answers/master_file.txt'
header_file = './data-challenge-1/Answers/wfirstColumnNumbers.txt'

rows = []
with open(master_file, "r") as f:
    for line in f:
        line = line.strip()
        # Skip empty lines or comment lines
        if not line or line.startswith("#"):
            continue

        tokens = line.split()  # split on whitespace
        # Keep only single-lens events
        if "dcnormffp" not in tokens:
            continue

        # Single-lens lines should have exactly 96 columns
        if len(tokens) != 96:
            continue

        rows.append(tokens)

df_sl = pd.DataFrame(rows)

# make an array of zeros with 97 elements
colnames_96 = np.zeros(96, dtype=object)

# Read the header file
with open(header_file, 'r') as f:
    for line in f:
        line = line.strip()
        # Skip empty lines or comments
        if not line or line.startswith('#'):
            continue
        # The second token is the 'name'
        parts = line.split()
        colnames_96[int(parts[0])] = parts[1]

#For single lenses they are (***Note for these, the mass of the lens is given by the planet mass column, not the host mass column):
#72 - unimportant
#73 - N, number of consecutive W149 data points deviating by >=3 sigma from a flat line
#74 - unimportant
#75 - Delta chi^2 (relative to a flat line)
#76-91 - unimportant
#92 - simulated event type (dcnormffp = single lens or free-floating planet)
#93 - unimportant (I think)
#94 - lightcurve filename root
#95 - Data challenge lightcurve number

# Replace the column names in colnames_96
colnames_96[73] = 'N'
colnames_96[75] = 'Delta chi2'
colnames_96[92] = 'sim type'
colnames_96[94] = 'filename'
colnames_96[95] = 'lc_number'

# Make sure the column names are unique
for i in range(94):
    if colnames_96[i] == '|' or colnames_96[i] == 0:
        colnames_96[i] = 'col_' + str(i)

# Replace the column names in the data_frame
df_sl.columns = colnames_96

# Remove the dummy columns 'col_*'
df_sl = df_sl.loc[:, ~df_sl.columns.str.startswith('col_')]

df_sl

,idx,subrun,field,l,b,ra,dec,src_id,Ds,Rs,...,sigma_q,sigma_rs,sigma_F00,sigma_fs0,sigma_F01,sigma_fs1,sigma_thetaE,sim type,filename,lc_number
0,1694,0,82,1.17028,-2.26944,269.319,-29.0889,9926,7.939,0.212,...,999999,999999,8905.98,76994.9,7414.03,3.44036e+09,-2.69363e+11,dcnormffp,dcnormffp_0_82_1694,5
1,539,0,82,1.05037,-2.20017,269.182,-29.158,16904,10.253,0.361,...,999999,999999,728415,435.739,50.4485,166721,-6.04003e+06,dcnormffp,dcnormffp_0_82_539,17
2,1278,0,82,1.18168,-2.1191,269.177,-29.0038,17351,10.64,0.375,...,999999,999999,2.78749e+06,629.296,7.91374,91347.9,-1.23727e+07,dcnormffp,dcnormffp_0_82_1278,21
3,1479,0,82,1.13373,-2.23737,269.267,-29.1045,8329,7.427,0.212,...,999999,999999,6.98775e+06,250.186,78.6551,59875.6,-979126,dcnormffp,dcnormffp_0_82_1479,22
4,318,0,82,1.08784,-2.21568,269.219,-29.1334,16773,10.169,0.535,...,999999,999999,3.11924e+06,2.92871,0.163627,1.18527,-215.015,dcnormffp,dcnormffp_0_82_318,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,718,0,82,1.17865,-2.19791,269.253,-29.0459,11807,8.458,0.293,...,999999,999999,592405,6337.05,10.9393,8.54477e+06,-1.21792e+09,dcnormffp,dcnormffp_0_82_718,285
70,1378,0,82,1.04452,-2.21048,269.189,-29.1682,2183,7.966,1.576,...,999999,999999,809.398,228.589,1.10915,21.9532,21943.9,dcnormffp,dcnormffp_0_82_1378,286
71,954,0,82,1.04501,-2.16747,269.147,-29.1462,14473,9.202,0.328,...,999999,999999,325492,21090.4,1237.92,1.09519e+08,-1.3671e+10,dcnormffp,dcnormffp_0_82_954,288
72,259,0,82,1.18813,-2.0865,269.149,-28.9819,10930,8.206,0.375,...,999999,999999,2.07119e+06,3929.98,34.1225,1.19947e+07,-5.83016e+08,dcnormffp,dcnormffp_0_82_259,290


The last column in this data frame has the lightcurve number, which we can use to pick out only the lightcurves matching our single-lens event list, for analysis.

In [20]:
#@title Figuring out which files we want
lc_number = df_sl['lc_number'].to_numpy()

lc_file_path_format = 'data-challenge-1/lc/ulwdc1_XXX_filter.txt'

lc_file_paths_W149 = [lc_file_path_format.replace('filter', 'W149')] * len(lc_number)
lc_file_paths_Z087 = [lc_file_path_format.replace('filter', 'Z087')] * len(lc_number)

# replace XXX, from the right, with the lc_number which is not necessarily of length 3
lc_file_paths_W149 = [path.replace('XXX', str(num).zfill(3)) for path, num in zip(lc_file_paths_W149, lc_number)]
lc_file_paths_Z087 = [path.replace('XXX', str(num).zfill(3)) for path, num in zip(lc_file_paths_Z087, lc_number)]

df_sl['lc_file_path_W149'] = lc_file_paths_W149
df_sl['lc_file_path_Z087'] = lc_file_paths_Z087

df_sl

,idx,subrun,field,l,b,ra,dec,src_id,Ds,Rs,...,sigma_F00,sigma_fs0,sigma_F01,sigma_fs1,sigma_thetaE,sim type,filename,lc_number,lc_file_path_W149,lc_file_path_Z087
0,1694,0,82,1.17028,-2.26944,269.319,-29.0889,9926,7.939,0.212,...,8905.98,76994.9,7414.03,3.44036e+09,-2.69363e+11,dcnormffp,dcnormffp_0_82_1694,5,data-challenge-1/lc/ulwdc1_005_W149.txt,data-challenge-1/lc/ulwdc1_005_Z087.txt
1,539,0,82,1.05037,-2.20017,269.182,-29.158,16904,10.253,0.361,...,728415,435.739,50.4485,166721,-6.04003e+06,dcnormffp,dcnormffp_0_82_539,17,data-challenge-1/lc/ulwdc1_017_W149.txt,data-challenge-1/lc/ulwdc1_017_Z087.txt
2,1278,0,82,1.18168,-2.1191,269.177,-29.0038,17351,10.64,0.375,...,2.78749e+06,629.296,7.91374,91347.9,-1.23727e+07,dcnormffp,dcnormffp_0_82_1278,21,data-challenge-1/lc/ulwdc1_021_W149.txt,data-challenge-1/lc/ulwdc1_021_Z087.txt
3,1479,0,82,1.13373,-2.23737,269.267,-29.1045,8329,7.427,0.212,...,6.98775e+06,250.186,78.6551,59875.6,-979126,dcnormffp,dcnormffp_0_82_1479,22,data-challenge-1/lc/ulwdc1_022_W149.txt,data-challenge-1/lc/ulwdc1_022_Z087.txt
4,318,0,82,1.08784,-2.21568,269.219,-29.1334,16773,10.169,0.535,...,3.11924e+06,2.92871,0.163627,1.18527,-215.015,dcnormffp,dcnormffp_0_82_318,29,data-challenge-1/lc/ulwdc1_029_W149.txt,data-challenge-1/lc/ulwdc1_029_Z087.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,718,0,82,1.17865,-2.19791,269.253,-29.0459,11807,8.458,0.293,...,592405,6337.05,10.9393,8.54477e+06,-1.21792e+09,dcnormffp,dcnormffp_0_82_718,285,data-challenge-1/lc/ulwdc1_285_W149.txt,data-challenge-1/lc/ulwdc1_285_Z087.txt
70,1378,0,82,1.04452,-2.21048,269.189,-29.1682,2183,7.966,1.576,...,809.398,228.589,1.10915,21.9532,21943.9,dcnormffp,dcnormffp_0_82_1378,286,data-challenge-1/lc/ulwdc1_286_W149.txt,data-challenge-1/lc/ulwdc1_286_Z087.txt
71,954,0,82,1.04501,-2.16747,269.147,-29.1462,14473,9.202,0.328,...,325492,21090.4,1237.92,1.09519e+08,-1.3671e+10,dcnormffp,dcnormffp_0_82_954,288,data-challenge-1/lc/ulwdc1_288_W149.txt,data-challenge-1/lc/ulwdc1_288_Z087.txt
72,259,0,82,1.18813,-2.0865,269.149,-28.9819,10930,8.206,0.375,...,2.07119e+06,3929.98,34.1225,1.19947e+07,-5.83016e+08,dcnormffp,dcnormffp_0_82_259,290,data-challenge-1/lc/ulwdc1_290_W149.txt,data-challenge-1/lc/ulwdc1_290_Z087.txt


There are a few pieces of information that may need to be known for each event that are not in the lightcurve files. These are stored in event_info.txt

Columns: `"Event_name"` `"Event_number"` `"RA_(deg)"` `"Dec_(deg)"` `"Distance"` `"A_W149"` `"sigma_A_W149"` `"A_Z087"` `"sigma_A_Z087"`

Distance, A_W149/Z087 are an estimate of the distance and extinction in each band of the red clump stars. sigma_A_W149/Z087 are dispersions in the extinction.

In [21]:
#@title Event information data frame

header = ["Event_name",
          "Event_number",
          "RA_(deg)",
          "Dec_(deg)",
          "Distance",
          "A_W149",
          "sigma_A_W149",
          "A_Z087",
          "sigma_A_Z087"
]

event_info = pd.read_csv('./data-challenge-1/event_info.txt', names=header, sep='\s+')
event_info

,Event_name,Event_number,RA_(deg),Dec_(deg),Distance,A_W149,sigma_A_W149,A_Z087,sigma_A_Z087
0,ulwdc1_001,1,269.165,-29.0207,8.18,0.73,0.01,1.41,0.01
1,ulwdc1_002,2,269.959,-30.1918,8.09,0.49,0.01,0.95,0.01
2,ulwdc1_003,3,269.100,-29.0983,8.18,0.73,0.01,1.41,0.01
3,ulwdc1_004,4,268.036,-28.3744,8.25,1.35,0.07,2.60,0.14
4,ulwdc1_005,5,269.319,-29.0889,8.18,0.73,0.01,1.41,0.01
...,...,...,...,...,...,...,...,...,...
288,ulwdc1_289,289,267.813,-28.6965,8.07,1.52,0.07,2.93,0.14
289,ulwdc1_290,290,269.149,-28.9819,8.18,0.73,0.01,1.41,0.01
290,ulwdc1_291,291,267.999,-29.7203,8.57,0.71,0.01,1.36,0.01
291,ulwdc1_292,292,269.151,-29.1433,8.18,0.73,0.01,1.41,0.01


In [22]:
#@title Combining the two data frames

# Convert 'lc_number' to numeric type before merging
merged_sl_df = pd.merge(event_info, df_sl.astype({'lc_number': 'int64'}), left_on='Event_number', right_on='lc_number', how='inner')
merged_sl_df

,Event_name,Event_number,RA_(deg),Dec_(deg),Distance,A_W149,sigma_A_W149,A_Z087,sigma_A_Z087,idx,...,sigma_F00,sigma_fs0,sigma_F01,sigma_fs1,sigma_thetaE,sim type,filename,lc_number,lc_file_path_W149,lc_file_path_Z087
0,ulwdc1_005,5,269.319,-29.0889,8.18,0.73,0.01,1.41,0.01,1694,...,8905.98,76994.9,7414.03,3.44036e+09,-2.69363e+11,dcnormffp,dcnormffp_0_82_1694,5,data-challenge-1/lc/ulwdc1_005_W149.txt,data-challenge-1/lc/ulwdc1_005_Z087.txt
1,ulwdc1_017,17,269.182,-29.1580,8.18,0.73,0.01,1.41,0.01,539,...,728415,435.739,50.4485,166721,-6.04003e+06,dcnormffp,dcnormffp_0_82_539,17,data-challenge-1/lc/ulwdc1_017_W149.txt,data-challenge-1/lc/ulwdc1_017_Z087.txt
2,ulwdc1_021,21,269.177,-29.0038,8.18,0.73,0.01,1.41,0.01,1278,...,2.78749e+06,629.296,7.91374,91347.9,-1.23727e+07,dcnormffp,dcnormffp_0_82_1278,21,data-challenge-1/lc/ulwdc1_021_W149.txt,data-challenge-1/lc/ulwdc1_021_Z087.txt
3,ulwdc1_022,22,269.267,-29.1045,8.18,0.73,0.01,1.41,0.01,1479,...,6.98775e+06,250.186,78.6551,59875.6,-979126,dcnormffp,dcnormffp_0_82_1479,22,data-challenge-1/lc/ulwdc1_022_W149.txt,data-challenge-1/lc/ulwdc1_022_Z087.txt
4,ulwdc1_029,29,269.219,-29.1334,8.18,0.73,0.01,1.41,0.01,318,...,3.11924e+06,2.92871,0.163627,1.18527,-215.015,dcnormffp,dcnormffp_0_82_318,29,data-challenge-1/lc/ulwdc1_029_W149.txt,data-challenge-1/lc/ulwdc1_029_Z087.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,ulwdc1_285,285,269.253,-29.0459,8.18,0.73,0.01,1.41,0.01,718,...,592405,6337.05,10.9393,8.54477e+06,-1.21792e+09,dcnormffp,dcnormffp_0_82_718,285,data-challenge-1/lc/ulwdc1_285_W149.txt,data-challenge-1/lc/ulwdc1_285_Z087.txt
70,ulwdc1_286,286,269.189,-29.1682,8.18,0.73,0.01,1.41,0.01,1378,...,809.398,228.589,1.10915,21.9532,21943.9,dcnormffp,dcnormffp_0_82_1378,286,data-challenge-1/lc/ulwdc1_286_W149.txt,data-challenge-1/lc/ulwdc1_286_Z087.txt
71,ulwdc1_288,288,269.147,-29.1462,8.18,0.73,0.01,1.41,0.01,954,...,325492,21090.4,1237.92,1.09519e+08,-1.3671e+10,dcnormffp,dcnormffp_0_82_954,288,data-challenge-1/lc/ulwdc1_288_W149.txt,data-challenge-1/lc/ulwdc1_288_Z087.txt
72,ulwdc1_290,290,269.149,-28.9819,8.18,0.73,0.01,1.41,0.01,259,...,2.07119e+06,3929.98,34.1225,1.19947e+07,-5.83016e+08,dcnormffp,dcnormffp_0_82_259,290,data-challenge-1/lc/ulwdc1_290_W149.txt,data-challenge-1/lc/ulwdc1_290_Z087.txt


### Binary Lens Events

In [23]:
rows = []
with open(master_file, "r") as f:
    for line in f:
        line = line.strip()
        # Skip empty lines or comment lines
        if not line or line.startswith("#"):
            continue

        tokens = line.split()  # split on whitespace
        # Keep only binary-lens events
        if "omcassan" not in tokens:
            continue

        # Single-lens lines should have exactly 96 columns
        if len(tokens) != 82:
            continue
        
        rows.append(tokens)
        
df_bl = pd.DataFrame(rows)

# For binary star lenses and bound planets they are:
# 72 - unimportant
# 73 - Delta chi^2 (relative to a flat line)
# 74 - unimportant
# 75 - Delta chi^2 (relative to a single lens fit PSPL or FSPL if necessary)
# 76 - |
# 77 - unimportant
# 78 - simulated event type (ombin = Binary star, omcassan = Bound planet) 
# 79 - unimportant (I think)
# 80 - lightcurve filename root
# 81 - Data challenge lightcurve number

# make an array of zeros with 82 elements
colnames_82 = np.zeros(82, dtype=object)

# Read the header file
with open(header_file, 'r') as f:
    for i, line in enumerate(f):
        if i >= 82:  # Only read the first 82 lines for the binary lens events
            break
        line = line.strip()
        # Skip empty lines or comments
        if not line or line.startswith('#'):
            continue
        # The second token is the 'name'
        parts = line.split()
        colnames_82[int(parts[0])] = parts[1]
        
# Replace the column names in colnames_82
colnames_82[73] = 'Delta chi2 flat'
colnames_82[75] = 'Delta chi2 single lens'
colnames_82[78] = 'sim type'
colnames_82[80] = 'filename'
colnames_82[81] = 'lc_number'

# Make sure the column names are unique
for i in range(82):
    if colnames_82[i] == '|' or colnames_82[i] == 0:
        colnames_82[i] = 'col_' + str(i)

# Replace the column names in the data_frame
df_bl.columns = colnames_82

# Remove the dummy columns 'col_*'
df_bl = df_bl.loc[:, ~df_bl.columns.str.startswith('col_')]
df_bl

,idx,subrun,field,l,b,ra,dec,src_id,Ds,Rs,...,flatsatFlag,flatchi2,Delta chi2 flat,chi2_1,Delta chi2 single lens,normw,sim type,sigma_tE,filename,lc_number
0,2683,4,87,1.21885,-0.937016,268.036,-28.3744,3589,9.675,0.886,...,0,|,76240,0,6860.52,|,omcassan,0.613786,omcassan_4_87_2683,4
1,262,2,83,1.04324,-2.03775,269.017,-29.0827,3730,9.003,0.688,...,0,|,380416,0,383.459,|,omcassan,0.338728,omcassan_2_83_262,8
2,657,3,32,0.133009,-1.76616,268.221,-29.7307,3803,9.215,1.225,...,0,|,4.23893e+07,0,5681.29,|,omcassan,0.404041,omcassan_3_32_657,12
3,623,0,8,-0.451107,-1.23271,267.348,-29.9601,7265,6.968,0.254,...,0,|,357404,0,2515.19,|,omcassan,0.105428,omcassan_0_8_623,25
4,1050,4,109,1.52853,-1.95811,269.216,-28.6232,4041,10.026,3.866,...,0,|,1.64356e+08,0,1218.02,|,omcassan,0.520323,omcassan_4_109_1050,32
5,833,3,88,1.11824,-0.668013,267.715,-28.3235,2281,8.093,0.861,...,0,|,955694,0,2718.51,|,omcassan,0.486863,omcassan_3_88_833,40
6,3146,4,18,-0.106723,-1.98473,268.298,-30.0481,3893,9.267,0.822,...,0,|,4.27037e+07,1,274.562,|,omcassan,0.540434,omcassan_4_18_3146,47
7,1873,2,34,0.0925122,-1.07923,267.518,-29.415,2979,8.895,0.822,...,0,|,241875,0,309.11,|,omcassan,0.294777,omcassan_2_34_1873,50
8,537,5,75,0.760366,-0.61572,267.455,-28.6039,15118,9.152,0.639,...,0,|,10863.4,0,364.734,|,omcassan,0.749276,omcassan_5_75_537,53
9,2947,3,9,-0.460921,-1.05033,267.162,-29.8747,18483,10.269,0.321,...,0,|,54087.3,0,1769.82,|,omcassan,0.495614,omcassan_3_9_2947,62


In [24]:
lc_number_bl = df_bl['lc_number'].to_numpy()
lc_file_path_format = 'data-challenge-1/lc/ulwdc1_XXX_filter.txt'
lc_file_paths_W149_bl = [lc_file_path_format.replace('filter', 'W149')] * len(lc_number_bl)
lc_file_paths_Z087_bl = [lc_file_path_format.replace('filter', 'Z087')] * len(lc_number_bl)
lc_file_paths_W149_bl = [path.replace('XXX', str(num).zfill(3)) for path, num in zip(lc_file_paths_W149_bl, lc_number_bl)]
lc_file_paths_Z087_bl = [path.replace('XXX', str(num).zfill(3)) for path, num in zip(lc_file_paths_Z087_bl, lc_number_bl)]
df_bl['lc_file_path_W149'] = lc_file_paths_W149_bl
df_bl['lc_file_path_Z087'] = lc_file_paths_Z087_bl
df_bl

,idx,subrun,field,l,b,ra,dec,src_id,Ds,Rs,...,Delta chi2 flat,chi2_1,Delta chi2 single lens,normw,sim type,sigma_tE,filename,lc_number,lc_file_path_W149,lc_file_path_Z087
0,2683,4,87,1.21885,-0.937016,268.036,-28.3744,3589,9.675,0.886,...,76240,0,6860.52,|,omcassan,0.613786,omcassan_4_87_2683,4,data-challenge-1/lc/ulwdc1_004_W149.txt,data-challenge-1/lc/ulwdc1_004_Z087.txt
1,262,2,83,1.04324,-2.03775,269.017,-29.0827,3730,9.003,0.688,...,380416,0,383.459,|,omcassan,0.338728,omcassan_2_83_262,8,data-challenge-1/lc/ulwdc1_008_W149.txt,data-challenge-1/lc/ulwdc1_008_Z087.txt
2,657,3,32,0.133009,-1.76616,268.221,-29.7307,3803,9.215,1.225,...,4.23893e+07,0,5681.29,|,omcassan,0.404041,omcassan_3_32_657,12,data-challenge-1/lc/ulwdc1_012_W149.txt,data-challenge-1/lc/ulwdc1_012_Z087.txt
3,623,0,8,-0.451107,-1.23271,267.348,-29.9601,7265,6.968,0.254,...,357404,0,2515.19,|,omcassan,0.105428,omcassan_0_8_623,25,data-challenge-1/lc/ulwdc1_025_W149.txt,data-challenge-1/lc/ulwdc1_025_Z087.txt
4,1050,4,109,1.52853,-1.95811,269.216,-28.6232,4041,10.026,3.866,...,1.64356e+08,0,1218.02,|,omcassan,0.520323,omcassan_4_109_1050,32,data-challenge-1/lc/ulwdc1_032_W149.txt,data-challenge-1/lc/ulwdc1_032_Z087.txt
5,833,3,88,1.11824,-0.668013,267.715,-28.3235,2281,8.093,0.861,...,955694,0,2718.51,|,omcassan,0.486863,omcassan_3_88_833,40,data-challenge-1/lc/ulwdc1_040_W149.txt,data-challenge-1/lc/ulwdc1_040_Z087.txt
6,3146,4,18,-0.106723,-1.98473,268.298,-30.0481,3893,9.267,0.822,...,4.27037e+07,1,274.562,|,omcassan,0.540434,omcassan_4_18_3146,47,data-challenge-1/lc/ulwdc1_047_W149.txt,data-challenge-1/lc/ulwdc1_047_Z087.txt
7,1873,2,34,0.0925122,-1.07923,267.518,-29.415,2979,8.895,0.822,...,241875,0,309.11,|,omcassan,0.294777,omcassan_2_34_1873,50,data-challenge-1/lc/ulwdc1_050_W149.txt,data-challenge-1/lc/ulwdc1_050_Z087.txt
8,537,5,75,0.760366,-0.61572,267.455,-28.6039,15118,9.152,0.639,...,10863.4,0,364.734,|,omcassan,0.749276,omcassan_5_75_537,53,data-challenge-1/lc/ulwdc1_053_W149.txt,data-challenge-1/lc/ulwdc1_053_Z087.txt
9,2947,3,9,-0.460921,-1.05033,267.162,-29.8747,18483,10.269,0.321,...,54087.3,0,1769.82,|,omcassan,0.495614,omcassan_3_9_2947,62,data-challenge-1/lc/ulwdc1_062_W149.txt,data-challenge-1/lc/ulwdc1_062_Z087.txt


In [25]:
event_info_bl = pd.read_csv('./data-challenge-1/event_info.txt', names=header, sep='\s+')
event_info_bl

,Event_name,Event_number,RA_(deg),Dec_(deg),Distance,A_W149,sigma_A_W149,A_Z087,sigma_A_Z087
0,ulwdc1_001,1,269.165,-29.0207,8.18,0.73,0.01,1.41,0.01
1,ulwdc1_002,2,269.959,-30.1918,8.09,0.49,0.01,0.95,0.01
2,ulwdc1_003,3,269.100,-29.0983,8.18,0.73,0.01,1.41,0.01
3,ulwdc1_004,4,268.036,-28.3744,8.25,1.35,0.07,2.60,0.14
4,ulwdc1_005,5,269.319,-29.0889,8.18,0.73,0.01,1.41,0.01
...,...,...,...,...,...,...,...,...,...
288,ulwdc1_289,289,267.813,-28.6965,8.07,1.52,0.07,2.93,0.14
289,ulwdc1_290,290,269.149,-28.9819,8.18,0.73,0.01,1.41,0.01
290,ulwdc1_291,291,267.999,-29.7203,8.57,0.71,0.01,1.36,0.01
291,ulwdc1_292,292,269.151,-29.1433,8.18,0.73,0.01,1.41,0.01


In [26]:
merged_bl_df = pd.merge(event_info_bl, df_bl.astype({'lc_number': 'int64'}), left_on='Event_number', right_on='lc_number', how='inner')
merged_bl_df

,Event_name,Event_number,RA_(deg),Dec_(deg),Distance,A_W149,sigma_A_W149,A_Z087,sigma_A_Z087,idx,...,Delta chi2 flat,chi2_1,Delta chi2 single lens,normw,sim type,sigma_tE,filename,lc_number,lc_file_path_W149,lc_file_path_Z087
0,ulwdc1_004,4,268.036,-28.3744,8.25,1.35,0.07,2.60,0.14,2683,...,76240,0,6860.52,|,omcassan,0.613786,omcassan_4_87_2683,4,data-challenge-1/lc/ulwdc1_004_W149.txt,data-challenge-1/lc/ulwdc1_004_Z087.txt
1,ulwdc1_008,8,269.017,-29.0827,8.20,0.66,0.01,1.27,0.01,262,...,380416,0,383.459,|,omcassan,0.338728,omcassan_2_83_262,8,data-challenge-1/lc/ulwdc1_008_W149.txt,data-challenge-1/lc/ulwdc1_008_Z087.txt
2,ulwdc1_012,12,268.221,-29.7307,8.57,0.71,0.01,1.36,0.01,657,...,4.23893e+07,0,5681.29,|,omcassan,0.404041,omcassan_3_32_657,12,data-challenge-1/lc/ulwdc1_012_W149.txt,data-challenge-1/lc/ulwdc1_012_Z087.txt
3,ulwdc1_025,25,267.348,-29.9601,8.53,1.62,0.04,3.11,0.07,623,...,357404,0,2515.19,|,omcassan,0.105428,omcassan_0_8_623,25,data-challenge-1/lc/ulwdc1_025_W149.txt,data-challenge-1/lc/ulwdc1_025_Z087.txt
4,ulwdc1_032,32,269.216,-28.6232,8.26,0.67,0.01,1.30,0.01,1050,...,1.64356e+08,0,1218.02,|,omcassan,0.520323,omcassan_4_109_1050,32,data-challenge-1/lc/ulwdc1_032_W149.txt,data-challenge-1/lc/ulwdc1_032_Z087.txt
5,ulwdc1_040,40,267.715,-28.3235,8.29,1.64,0.15,3.15,0.30,833,...,955694,0,2718.51,|,omcassan,0.486863,omcassan_3_88_833,40,data-challenge-1/lc/ulwdc1_040_W149.txt,data-challenge-1/lc/ulwdc1_040_Z087.txt
6,ulwdc1_047,47,268.298,-30.0481,8.40,0.62,0.01,1.19,0.01,3146,...,4.27037e+07,1,274.562,|,omcassan,0.540434,omcassan_4_18_3146,47,data-challenge-1/lc/ulwdc1_047_W149.txt,data-challenge-1/lc/ulwdc1_047_Z087.txt
7,ulwdc1_050,50,267.518,-29.4150,8.41,1.33,0.04,2.56,0.08,1873,...,241875,0,309.11,|,omcassan,0.294777,omcassan_2_34_1873,50,data-challenge-1/lc/ulwdc1_050_W149.txt,data-challenge-1/lc/ulwdc1_050_Z087.txt
8,ulwdc1_053,53,267.455,-28.6039,8.39,2.32,0.24,4.47,0.47,537,...,10863.4,0,364.734,|,omcassan,0.749276,omcassan_5_75_537,53,data-challenge-1/lc/ulwdc1_053_W149.txt,data-challenge-1/lc/ulwdc1_053_Z087.txt
9,ulwdc1_062,62,267.162,-29.8747,8.57,2.20,0.20,4.23,0.38,2947,...,54087.3,0,1769.82,|,omcassan,0.495614,omcassan_3_9_2947,62,data-challenge-1/lc/ulwdc1_062_W149.txt,data-challenge-1/lc/ulwdc1_062_Z087.txt


In [27]:
try:  # this will only work on Colab.
  sl_sheet = sheets.InteractiveSheet(df=merged_sl_df)
  sl_sheet.show()
  bl_sheet = sheets.InteractiveSheet(df=merged_bl_df)
  bl_sheet.show()
except:
  pass

Great - data successfully wrangled. Let's forget we ever had to live through that and move right along.

## Packages Covered in This Notebook

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;">

<details>
<summary><font face="Helvetica" size="5">1. MulensModel</font></summary>

[Go to the `MulensModel` section](#mulensmodel)
  * [1.1 Data objects](#11-data-objects)
  * [1.2 Model objects](#12-model-objects)
  * [1.3 Event objects](#13-event-objetcs)
  * [1.4 Fitting](#fitting-framework)
  * [1.5 Higher-order effects](#model-examples)
    - [1S1L](#1s1l-example)
    - [1S2L](#1s2l-example)
    - [1S3L](#1s3l-example)
    - [2S1L](#2s1l-example)
    - [Parallax](#parallax-example)
</details>
<details>
<summary><font face="Helvetica" size="5">2. pyLIMA</font></summary>

[Go to the `pyLIMA` section](#pylima)
  * [2.1 Installation and Setup](#pylima-installation)
  * [2.2 Basic Usage](#pylima-basic)
  * [2.3 Model Examples](#pylima-models)
    - [2.3.1 Single Lens (PSPL)](#pylima-pspl)
    - [2.3.2 Finite Source (FSPL)](#pylima-fspl)
    - [2.3.3 Simulation Capabilities](#pylima-simulation)
  * [2.4 Advanced Features](#pylima-advanced)
    - [2.4.1 Custom Parameter Definitions](#pylima-custom-params)
    - [2.4.2 Multiple Telescopes and Filters](#pylima-multiple-telescopes)
  * [2.5 Performance and Best Practices](#pylima-performance)
</details>
<details>
<summary><font face="Helvetica" size="5">3. RTModel</font></summary>

[Go to the `RTModel` section](#rtmodel)
  * [3.1 Installation and Setup](#rtmodel-installation)
  * [3.2 Data Preparation](#rtmodel-data-prep)
  * [3.3 Basic Usage](#rtmodel-basic-usage)
  * [3.4 Understanding RTModel Output](#rtmodel-output)
  * [3.5 Model Categories](#rtmodel-model-categories)
  * [3.6 Visualization and Results](#rtmodel-visualization)
  * [3.7 Advanced Features](#rtmodel-advanced)
  * [3.8 RTModel vs Other Tools](#rtmodel-comparison)
  * [3.9 Astrophotometric Fitting (RTModel v3.0)](#rtmodel-astrometric)
    - [3.9.1 Astrophotometric Data Format](#rtmodel-astrometric-data)
    - [3.9.2 Astrophotometric Parameters](#rtmodel-astrometric-parameters)
    - [3.9.3 Astrophotometric Fitting](#rtmodel-astrometric-fitting)
    - [3.9.4 Astrometric Visualization](#rtmodel-astrometric-visualization)
    - [3.9.5 Applications and Use Cases](#rtmodel-astrometric-applications)
</details>
<details>
<summary><font face="Helvetica" size="5">4. popclass</font></summary>

[Go to the `popclass` section](#popclass)
  * [4.1 Installation](#popclass-installation)
  * [4.2 Basic Usage Example](#popclass-basic-usage)
  * [4.3 How It Works](#popclass-theory)
  * [4.4 Population Models](#popclass-models)
  * [4.5 Advanced Features](#popclass-advanced)
  * [4.6 Best Practices & Caveats](#popclass-best-practices)
  * [4.7 Further Reading](#popclass-references)
</details>
<!--
<details>
<summary><font face="Helvetica" size="5">5. BAGEL</font></summary> -->
<!--
[Go to the `BAGEL` section](#bagel)
</details>
<details>
<summary><font face="Helvetica" size="5">6. MuLAN</font></summary> -->
<!--
[Go to the `MuLAN` section](#mulan)
</details> -->

## 1. MulensModel

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;">

`MulensModel` is arguably the simplest and most well-known microlensing modelling tool. It is an extremely capable tool, having a vast array of models and higher-order effects, but the developers do not attempt to automate the model selection process or infer physical parameter, as of yet.


In [ ]:
#@title Importing the package
import MulensModel as mm

### 1.1 Data Objects

When fitting Roman data with `MulensModel`, we need to make a minor adjustment to our model for data that is not ground based. In `MulensModel` this simply means adding the following keyword to the data object initialization: `ephemerides_file=PATH_TO_THE_FILE`.

> Instructions specific to this data set for `MulensModel` are given [here](https://github.com/rpoleski/MulensModel/blob/master/documents/data_challenge.md).

Most of the data for these events is in the W147 band, so we make the very reasonable decision to just fit those data and not have to deal with mutliple data sets with different $F_\textrm{S}$ and $F_\textrm{B}$ values. That should speen up our fits too. If we wanted to find the color of the source star at a later date we could fit just the flux parameters and leave the microlensing-model parameters fixed (as described in [this notebook](https://amberlee2427.github.io/TheMicrolensersGuideToTheGalaxy/Solved/SingleLens.html)) using a linear regression, which would take a fraction of a second per event.

In [ ]:
#@title Collecting the relevent meta data
event_idx = 0
data_file = merged_sl_df['lc_file_path_W149'][event_idx]
ra = merged_sl_df['RA_(deg)'][event_idx]
dec = merged_sl_df['Dec_(deg)'][event_idx]
print(ra, dec)

# convert decimal ra and dec in degrees to "17h57m16.56s -29d05m20.04s"
coord = SkyCoord(ra=ra * u.deg, dec=dec * u.deg, frame='icrs')
hms_dms_string = coord.to_string('hmsdms')
print(f"SkyCoord default: {hms_dms_string}")

In [ ]:
#@title Example L2 `Data` object
# Here is the main difference for space data - we provide the ephemeris for Roman:
EPHEM_FILE = 'data-challenge-1/wfirst_ephemeris_W149.txt'
data_Roman_W149 = mm.MulensData(file_name=data_file,
                                phot_fmt='mag',
                                ephemerides_file=EPHEM_FILE,
                                plot_properties={'color': '#a859e4',
                                                 'label': 'Roman W149'
                                                 },
                                bandpass='H'
                               )

### 1.2 Model Objects

`Model` objects store the parameters (e.g. `t_0`) being fitted and any meta-parameters (e.g. `t_0_par`; the parallax reference time) that are required. These parameters decide what kind of microlensing model and higher-order effects will be simulated (e.g. a single-lens or binary-lens event).

In [ ]:
#@title Collecting the meta data
# split the parallax equally out of a lack of better ideas
piE_truth = float(merged_sl_df['piE'][event_idx])
pi_E_E = np.sqrt(piE_truth**2 / 2.0)
pi_E_N = pi_E_E * 1.0

t_0 = float(merged_sl_df['t0'][event_idx])
# Annoyingly, t_0 is in simulation time not HJD so we need to do a conversion
# (https://github.com/microlensing-data-challenge/evaluation_code/blob/master/parse_table1.py)
# line 402
t_0_truth = t_0 + 2458233.5  # adding simulation 0 time - 0.5 days (I'm not sure why we need the MJD/BJD correction)

In [ ]:
#@title Making a dictionary of the guess parameters
# Let's just tidy the "guess" parameters up into a dictionary, for easy accesss
params = dict()
parameters_to_fit = ["t_0", "u_0", "t_E", "rho", "pi_E_N", "pi_E_E"]
params['t_0'] = t_0_truth
params['t_0_par'] = t_0_truth
u_0_truth = float(merged_sl_df['u0'][event_idx])
params['u_0'] = u_0_truth
t_E_truth = float(merged_sl_df['tE'][event_idx])
params['t_E'] = t_E_truth * 1.1
rho_truth = float(merged_sl_df['rhos'][event_idx])
params['rho'] = rho_truth * 1.1
params['pi_E_N'] = pi_E_E
params['pi_E_E'] = pi_E_E
print(params)

In [ ]:
#@title Example `Model` object
# If we are using parallax, it is also important that we provide the event
# coordinates, or MulensModel can't do necessary calculations
Roman_guess_model = mm.Model({**params},
                        coords=coord,
                        ephemerides_file=EPHEM_FILE
                       )

### 1.3 Event Objects

Event objects store the model we defined earlier and the data, together.

In [ ]:
#@title Example `Event` object
Roman_event = mm.Event(datasets=data_Roman_W149, model=Roman_guess_model)

### 1.4 Fitting Framework

Now we'll set up a comprehensive fitting framework that can handle different model types, so that we can compare some optimizing algorithms and demonstrate various microlensing models, later in this section.

In [ ]:
#@title Set up plotting style
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

In [ ]:
#@title Define likelihood function for fitting
def log_likelihood(theta, event, parameters_to_fit):
    """Log likelihood function for emcee"""

    penalty = 0.0

    # --- soft priors / sanity bounds ---
    log_rho = None
    if 'logrho' in parameters_to_fit:
        rho_index = parameters_to_fit.index('logrho')
        log_rho = theta[rho_index]
    elif 'log_rho' in parameters_to_fit:
        rho_index = parameters_to_fit.index('log_rho')
        log_rho = theta[rho_index]
    elif 'rho' in parameters_to_fit:
        rho_index = parameters_to_fit.index('rho')
        if theta[rho_index] <= 0.0:
            return -np.inf
        log_rho = np.log10(theta[rho_index])

    if log_rho is not None and log_rho > 0.0:
        penalty += (log_rho / 0.5) ** 2  # Penalize unphysically large rho

    # robust t_E extraction (handles logt_E naming variants)
    tE = None
    if 'log_t_E' in parameters_to_fit:
        tE = 10**theta[parameters_to_fit.index('log_t_E')]
    elif 'logt_E' in parameters_to_fit:
        tE = 10**theta[parameters_to_fit.index('logt_E')]
    elif 'log_tE' in parameters_to_fit:
        tE = 10**theta[parameters_to_fit.index('log_tE')]
    elif 'logtE' in parameters_to_fit:
        tE = 10**theta[parameters_to_fit.index('logtE')]
    elif 't_E' in parameters_to_fit:
        tE = theta[parameters_to_fit.index('t_E')]
    elif 'tE' in parameters_to_fit:
        tE = theta[parameters_to_fit.index('tE')]
    else:
        # not being fit; use current model value
        tE = float(event.model.parameters.t_E)

    if tE <= 0.0 or not np.isfinite(tE):
        return -np.inf  # hard bound on non-physical tE

    # robust u0 extraction
    if 'u_0' in parameters_to_fit:
        u0 = theta[parameters_to_fit.index('u_0')]
    elif 'u0' in parameters_to_fit:
        u0 = theta[parameters_to_fit.index('u0')]
    else:
        u0 = float(event.model.parameters.u_0)

    if abs(u0) > 1.0:
        penalty += (u0 / 3.0) ** 2  # Penalize unphysically large u_0

    # q bounds (only when binary parameters are part of fit)
    q = None
    if 'logq' in parameters_to_fit:
        q = 10**theta[parameters_to_fit.index('logq')]
    elif 'log_q' in parameters_to_fit:
        q = 10**theta[parameters_to_fit.index('log_q')]
    elif 'q' in parameters_to_fit:
        q = theta[parameters_to_fit.index('q')]
    if q is not None:
        if q <= 0.0:
            return -np.inf  # hard bound on negative q
        if q > 1.0:
            return -np.inf  # hard bound on q > 1

    # Update model parameters (all logs are base 10)
    for i, param_name in enumerate(parameters_to_fit):
        if param_name == 'log_rho' or param_name == 'logrho':
            event.model.parameters.rho = 10**theta[i]
        elif param_name == 'log_t_E' or param_name == 'logt_E' or param_name == 'logtE' or param_name == 'log_tE':
            event.model.parameters.t_E = 10**theta[i]
        elif param_name == 'logs' or param_name == 'log_s' or param_name == 'logs':
            event.model.parameters.s = 10**theta[i]
        elif param_name == 'logq' or param_name == 'log_q' or param_name == 'logq':
            event.model.parameters.q = 10**theta[i]
        else:
            setattr(event.model.parameters, param_name, theta[i])

    # Fit the fluxes given the current model parameters
    event.fit_fluxes()

    # Get the source and blend fluxes
    ([F_S], F_B) = event.get_flux_for_dataset(event.datasets[0])

    # Calculate chi-squared
    chi2 = event.get_chi2()

    # Add flux priors if needed (optional)
    if F_B <= 0:
        penalty += ((F_B / 10) ** 2)  # Penalize negative blend flux
    if F_S <= 0 or (F_S + F_B) <= 0:
        return -np.inf  # Return inf if fluxes are non-physical

    return -0.5 * chi2 - penalty

def neg_log_likelihood(params_array, event, parameters_to_fit):
    """Negative log likelihood function for scipy.optimize.minimize"""

    return -log_likelihood(params_array, event, parameters_to_fit)

def params2flux(theta, t, event, parameters_to_fit):
    """Helper function to convert parameters to fluxes for curve_fit"""
    for i, param_name in enumerate(parameters_to_fit):
        if param_name == 'log_rho' or param_name == 'logrho':
            # Convert log_rho back to rho
            event.model.parameters.rho = 10**theta[i]
        elif param_name == 'log_t_E' or param_name == 'logt_E' or param_name == 'logtE' or param_name == 'log_tE':
            event.model.parameters.t_E = 10**(theta[i])
        elif param_name == 'tE' or param_name == 't_E':
            event.model.parameters.t_E = np.abs(theta[i])
        elif param_name == 'logs' or param_name == 'log_s' or param_name == 'logs':
            event.model.parameters.s = 10**theta[i]
        elif param_name == 's':
            event.model.parameters.s = np.abs(theta[i])
        elif param_name == 'logq' or param_name == 'log_q' or param_name == 'logq':
            event.model.parameters.q = 10**theta[i]
        elif param_name == 'q':
            if theta[i] <= 0.0:
                event.model.parameters.q = np.abs(theta[i]) + 1e-20  # tricky little bound on negative q
            if theta[i] > 1.0:
                event.model.parameters.q = 1.0/theta[i]  # tricky little bound on q > 1
        else:
            setattr(event.model.parameters, param_name, theta[i])
    ([F_S], F_B) = event.get_flux_for_dataset(event.datasets[0])

    return event.model.get_magnification(t) * F_S + F_B

def finite_difference_hessian(func, x, step_scale=1e-4):
    x = np.asarray(x, dtype=float)
    n = x.size
    h = step_scale * np.maximum(1.0, np.abs(x))
    H = np.zeros((n, n), dtype=float)
    f0 = func(x)

    for i in range(n):
        ei = np.zeros(n); ei[i] = h[i]
        f_ip = func(x + ei)
        f_im = func(x - ei)
        H[i, i] = (f_ip - 2.0 * f0 + f_im) / (h[i] ** 2)

        for j in range(i + 1, n):
            ej = np.zeros(n); ej[j] = h[j]
            f_pp = func(x + ei + ej)
            f_pm = func(x + ei - ej)
            f_mp = func(x - ei + ej)
            f_mm = func(x - ei - ej)
            Hij = (f_pp - f_pm - f_mp + f_mm) / (4.0 * h[i] * h[j])
            H[i, j] = Hij
            H[j, i] = Hij

    return H

def run_mcmc_fit(event, parameters_to_fit, initial_guess, nwalkers=32, nsteps=1000, burnin=200, n_cores=None):
    ndim = len(parameters_to_fit)
    pos = np.array(initial_guess) + 1e-4 * np.random.randn(nwalkers, ndim)

    if n_cores is None:
        n_cores = min(nwalkers, os.cpu_count() or 1)
    else:
        n_cores = min(n_cores, nwalkers)

    print(f"Using {n_cores} worker processes for MCMC")

    with Pool(nodes=n_cores) as pool:  # pathos ProcessingPool -> nodes, not processes
        sampler = emcee.EnsembleSampler(
            nwalkers, ndim, log_likelihood,
            args=(event, parameters_to_fit),
            pool=pool
        )
        sampler.run_mcmc(pos, nsteps, progress=True)

    return sampler

### 1.5 Model Examples

Now let's demonstrate different microlensing models. We'll start with simple models and work our way up to more complex ones.

#### 1.5.1 Single Lens Single Source (1S1L) with Finite Source Effects

Let's jump straight in to simple single lens model that includes the finite source effect (ρ parameter) and parallax. These are important higher-order effects for breaking the microlensing mass-distance degeneracy, so it is highly useful to know how to include them in your models. Functionally, the finite source effect smooths over sharp features in the light-curve and parallax changes the apparent trajectory of the source (relative to the lens) as the observer moves around the solar-system. In practice, parallax is oftem not well constrained from a single observer over typical event timescales. 

The finite source effect adds an additional parameter to the generative model, usualy called ρ (`rho`). This is the size of the source star in units of Einstein angle (θᴇ).

Parallax adds two new parameter to the generative model (for `MulensModel` these are the north and east components of the Einstsein parallax, πᴇ; `pi_E_N`, `pi_E_E`) and one meta-parameter that defines the inertial frame (`t_0_par`).

> The example data we are using has only the Einstein parallax magnitude, and not it's direction, so we will make poorly informed guesses at how to split these components. 

In [ ]:
#@title 1S1L Model Setup
# Select a single lens event for demonstration
gamma = float(merged_sl_df['gamma'][event_idx])

print(f"Event {event_idx}: Truth parameters")
print(f"  t_0: {t_0_truth:.3f}")
print(f"  u_0: {u_0_truth:.4f}")
print(f"  tᴇ: {t_E_truth:.2f} days")
print(f"  ρ: {rho_truth:.6f}")
print(f"  πᴇ: {piE_truth:.2f} mas")
print(f"  Γ: {gamma:.4f}")

# Finite source effects parameter:
# - rho: angular source radius normalized to the Einstein radius (θ_*/θ_E)

# Parallax parameters:
# - pi_E_N: North component of parallax
# - pi_E_E: East component of parallax
# - t_0_par: reference time for parallax (usually same as t_0). This is the
#   time at which parallax does not purturb the relative source trajectory.

In [ ]:
#@title Load data for 1S1L example (teaching-first version)
# Optional controls
FIT_WINDOW_IN_TE = 3.0      # fit only data within t0 ± FIT_WINDOW_IN_TE * tE (it's handy to have a good guess for tE)
USE_CROPPED_DATA = True      # set False to use full light curve (slower)

# 1) Load data
data_1s1l = data_Roman_W149.copy()

# 2) Initial parameter guess
params_1s1l = {}  # Model object requires linear parameters
log_params_1s1l = {}  # but we will be sampling in log space for t_E and rho
parameters_to_fit_1s1l = ["t_0", "u_0", "log_t_E", "log_rho", "pi_E_N", "pi_E_E"]
for param_name in parameters_to_fit_1s1l:
    linear_param_name = param_name.replace('log_', '').replace('log','')
    if linear_param_name == param_name:
        params_1s1l[linear_param_name] = params[linear_param_name]
        log_params_1s1l[param_name] = params[linear_param_name]
    else:
        params_1s1l[linear_param_name] = params[linear_param_name]
        log_params_1s1l[param_name] = np.log10(params[linear_param_name])
print(log_params_1s1l)
print(params_1s1l)

# 3) Build model and resolve warnings
model_1s1l = mm.Model(params_1s1l, coords=coord, ephemerides_file=EPHEM_FILE)
model_1s1l.set_limb_coeff_gamma('H', float(gamma))  # resolves H-band limb-darkening warning

# Use finite-source only near peak; point-source outside (faster)
rho =  max(rho_truth, 1e-6)
dt_fs = max(0.5, 5.0 * rho * t_E_truth)
fs_method = 'finite_source_uniform_Gould94' if abs(float(gamma)) < 1e-8 else 'finite_source_LD_Yoo04'
if USE_CROPPED_DATA:
    t_min, t_max = t_0_truth - FIT_WINDOW_IN_TE * t_E_truth, t_0_truth + FIT_WINDOW_IN_TE * t_E_truth
else:
    t_min = np.min(data_1s1l.time) - 5.0
    t_max = np.max(data_1s1l.time) + 5.0
# https://rpoleski.github.io/MulensModel/MulensModel.magnificationcurve.html#MulensModel.magnificationcurve.MagnificationCurve.get_point_lens_magnification
model_1s1l.set_magnification_methods([
    t_min, 
    'point_source', 
    t_0_truth - dt_fs, 
    fs_method, 
    t_0_truth + dt_fs, 
    'point_source',
    t_max
])

# 4) Build event
event_1s1l = mm.Event(datasets=data_1s1l, model=model_1s1l)

# 5) Optional time-window crop for speed
if USE_CROPPED_DATA:
    m = (data_1s1l.time >= t_min) & (data_1s1l.time <= t_max)
    data_1s1l_fit = mm.MulensData(
        data_list=[data_1s1l.time[m], data_1s1l.mag[m], data_1s1l.err_mag[m]],
        phot_fmt='mag',
        ephemerides_file=EPHEM_FILE,
        plot_properties={'color': '#a859e4', 'label': 'Roman W149 (cropped)'},
        bandpass='H',
    )
    event_1s1l = mm.Event(datasets=data_1s1l_fit, model=model_1s1l)
    print(f'Using cropped data: {m.sum()} points in [{t_min:.2f}, {t_max:.2f}]')
else:
    print(f'Using full data: {len(data_1s1l.time)} points')

In [ ]:
# Plot the initial model
plt.figure(figsize=(12, 6))
event_1s1l.plot_data()
# Get the source and blend fluxes
#event_1s1l.fit_fluxes()
#([F_S], F_B) = event_1s1l.get_flux_for_dataset(event_1s1l.datasets[0])
#event_1s1l.model.plot_lc(source_flux=F_S, blend_flux=F_B, color='black', label='Initial Model')
event_1s1l.plot_model(color='black', label='Initial Model')
plt.title(f"1S1L Event {event_idx} - Initial Model")
plt.xlabel("BJD")
plt.ylabel("Magnitude")
plt.xlim(2459810.0, 2459822.0)
plt.legend()
plt.show()
#plt.savefig('1s1l_initial.png')
print(event_1s1l.model.parameters)

In [ ]:
#@title Fit 1S1L model with MCMC
# Set up parameters to fit
initial_guess_1s1l = list(log_params_1s1l.values())
print(initial_guess_1s1l)
truths_transformed = [
    t_0_truth,
    u_0_truth,
    t_E_truth,
    np.log10(rho_truth),
    np.sqrt(piE_truth**2/2.0),
    np.sqrt(piE_truth**2/2.0)
]

In [ ]:
#@title Sanity Check
print("event_idx:", event_idx)
print("data_file:", data_file)
print("time span:", float(np.min(data_1s1l.time)), float(np.max(data_1s1l.time)))
print("t0 truth/model:", t_0_truth, params_1s1l["t_0"])
print("u0 truth/model:", u_0_truth, params_1s1l["u_0"])
print("tE truth/model:", t_E_truth, params_1s1l["t_E"])

##### A Short Aside to Compare Optimizers

You typically find that the go to optimization method when fitting microlensing events is MCMC, but this isn't aways the ideal choice. MCMC is extremely robust and able to convey complex poterior covariences, but there are times when it is overkill.

In [ ]:
print("Starting 1S1L MCMC fit...")
start_time = time.time()

# Run MCMC
# burnin must be less than nsteps, because n posterior samples is nsteps - burnin
sampler_1s1l = run_mcmc_fit(event_1s1l, parameters_to_fit_1s1l, initial_guess_1s1l,
                           nwalkers=32, nsteps=500, burnin=250)  
end_time = time.time()
print(f"\n1S1L MCMC fit completed in {(end_time - start_time)/60:.2f} minutes")

print("\nResults:")
# Get results
samples_1s1l = sampler_1s1l.chain[:, 100:, :].reshape((-1, len(parameters_to_fit_1s1l)))
best_fit_1s1l = np.median(samples_1s1l, axis=0)
uncertainties_1s1l = np.std(samples_1s1l, axis=0)
mcmc_linear_sol = {}
for i, param in enumerate(parameters_to_fit_1s1l):
    print(f"  {param}: {best_fit_1s1l[i]:.6f} ± {uncertainties_1s1l[i]:.6f}")
# Compare with truth
print("\nComparison with truth:")
for i, param in enumerate(parameters_to_fit_1s1l):
    diff = best_fit_1s1l[i] - truths_transformed[i]
    sigma_diff = abs(diff) / uncertainties_1s1l[i]
    print(f"  {param}: {diff:.6f} ({sigma_diff:.2f}σ)")
    # Convert best fit parameters back to physical units for plotting
    if 'log' in param:
        physical_value = 10**best_fit_1s1l[i]
        label = param.replace('log_', '').replace('log', '')
    else:
        physical_value = best_fit_1s1l[i]
        label = param
    mcmc_linear_sol[label] = physical_value

This isn't enough samples for a well-sampled posterior distribution as we will see later, but this is just a demonstration, so we'll keep it short to save time. Ideally you would keep sampling until your walkers are no longer exploring new areas of parameter space and (have settled into a χ² trough) and are mixing well. You would throw every sample before this point away and sample the posterior distribution of this minima you have discovered until your corner plot histograms look smooth.

If you are looking for speed, you could consider strategically binning your data, truncating it, or trying-out a different optimizing algorithm like Nelder-Mead using `scipy.optimize.minimize`, which will be much faster for a simple 1s1l fit.

In [ ]:
print("Starting 1S1L Nelder-Mead fit...")
start_time = time.time()

# Nelder-Mead can need many evaluations; increase limits and tighten tolerances
nm_options = {
    'maxiter': 4000,
    'maxfev': 12000,
    'xatol': 1e-5,
    'fatol': 1e-4,
    'adaptive': True,
}

with warnings.catch_warnings():  # catch noisy warnings when trial points are out of range
    warnings.filterwarnings("ignore", category=ErfaWarning)
    result = minimize(
        neg_log_likelihood,
        initial_guess_1s1l,
        args=(event_1s1l, parameters_to_fit_1s1l),
        method='Nelder-Mead',
        options=nm_options,
    )

nm_res = result.x
print(result)
if not result.success:
    print("\n⚠️ Nelder-Mead did not fully converge; uncertainties may be unreliable.")

end_time = time.time()
print(f"\n1S1L Nelder-Mead fit completed in {(end_time - start_time)/60:.2f} minutes")

# But what about parameter uncertainties? we can numerically estimate the Hessian at the best fit point 
# to get an estimate of the covariance matrix

# Estimate covariance from a numerical Hessian of the scalar objective at the best fit
print("Calculating parameter uncertainties from numerical Hessian...")
start_time = time.time()

def objective(theta):
    return neg_log_likelihood(theta, event_1s1l, parameters_to_fit_1s1l)

hessian = finite_difference_hessian(objective, nm_res)

# Diagnose ill-conditioning (flat/degenerate directions -> unreliable uncertainties)
hessian_cond = np.linalg.cond(hessian)
print(f"Hessian condition number: {hessian_cond:.2e}")
if (not np.isfinite(hessian_cond)) or (hessian_cond > 1e12):
    print("⚠️ Hessian is ill-conditioned; some 1σ errors are not trustworthy.")

# Use pseudo-inverse for numerical stability
cov_matrix = np.linalg.pinv(hessian)
diag = np.diag(cov_matrix).astype(float)

# Mark non-physical/unstable variances as NaN instead of forcing 0
diag[~np.isfinite(diag)] = np.nan
diag[diag <= 0] = np.nan
perr = np.sqrt(diag)

end_time = time.time()
print(f"Parameter uncertainty calculation completed in {(end_time - start_time)/60:.2f} minutes")

print("\nResults:")
nm_linear_sol = {}
for i, param in enumerate(parameters_to_fit_1s1l):
    err_txt = f"{perr[i]:.6f}" if np.isfinite(perr[i]) else "nan"
    print(f"  {param}: {nm_res[i]:.6f} ± {err_txt}")

print("\nComparison with truth:")
linear_nm_res = nm_res.copy()
for i, param in enumerate(parameters_to_fit_1s1l):
    diff = nm_res[i] - truths_transformed[i]
    if np.isfinite(perr[i]) and perr[i] > 0:
        sigma_diff = abs(diff) / perr[i]
        sigma_txt = f"{sigma_diff:.2f}σ"
    else:
        sigma_txt = "n/a (undefined uncertainty)"
    print(f"  {param}: {diff:.6f} ({sigma_txt})")
    # Convert best fit parameters back to physical units for plotting
    if 'log' in param:
        physical_value = 10**nm_res[i]
        label = param.replace('log_', '').replace('log', '')
    else:
        physical_value = nm_res[i]
        label = param
    nm_linear_sol[label] = physical_value

Another option to consider are gradient descent menthods such as `scipy.optimize.curve_fit`. For 1s1l fits, this methods is ideal because you get parameter uncertainties for free from the covariance matrix, and you don't have to worry about convergence or burn-in. However constraining the fit get's a little tricky, and it can be sensitive to initial guesses, but it's very fast for simple models like 1s1l.

> Worth noting are the `Event.get_chi2_gradient(parameters_to_fit)` and `Event.calculate_chi2_gradient()` functions which could be of use for graident descent optimizers. Make sure to set the model parameters before calling. 

In [ ]:
print("Starting 1S1L curve-fit...")
start_time = time.time()

# Use the same dataset currently attached to event_1s1l (cropped or full)
fit_data = event_1s1l.datasets[0]
x_fit = fit_data.time
y_fit = fit_data.flux
sigma_fit = fit_data.err_flux.copy()

# Guard against bad/zero uncertainties
good_sigma = np.isfinite(sigma_fit) & (sigma_fit > 0)
if not np.any(good_sigma):
    raise ValueError("No valid flux uncertainties available for curve_fit.")
sigma_floor = np.nanmedian(sigma_fit[good_sigma])
sigma_fit[~good_sigma] = sigma_floor

# Build conservative bounds to prevent unphysical trial points
lb, ub = [], []
for p in parameters_to_fit_1s1l:
    if p == 't_0':
        lb.append(t_0_truth - 20.0); ub.append(t_0_truth + 20.0)
    elif p in ('u_0', 'u0'):
        lb.append(-2.0); ub.append(2.0)
    elif p in ('t_E', 'tE'):
        lb.append(0.05); ub.append(500.0)
    elif p in ('log_t_E', 'log_tE', 'logt_E', 'logtE'):
        lb.append(0.05); ub.append(500.0)
    elif p in ('logrho', 'log_rho'):
        lb.append(-7.0); ub.append(-0.3)
    elif p == 'rho':
        lb.append(1e-7); ub.append(0.5)
    elif p in ('pi_E_E', 'pi_E_N'):
        lb.append(-5.0); ub.append(5.0)
    else:
        lb.append(-np.inf); ub.append(np.inf)
lb = np.array(lb, dtype=float)
ub = np.array(ub, dtype=float)

# Ensure p0 is strictly inside bounds for TRF
p0 = np.array(initial_guess_1s1l, dtype=float)
eps = 1e-10
finite_lb = np.isfinite(lb)
finite_ub = np.isfinite(ub)
p0[finite_lb] = np.maximum(p0[finite_lb], lb[finite_lb] + eps)
p0[finite_ub] = np.minimum(p0[finite_ub], ub[finite_ub] - eps)

# curve_fit expects: f(x, *params)
def model_flux(t, *theta):
    try:
        flux = params2flux(theta, t, event_1s1l, parameters_to_fit_1s1l)
        if not np.all(np.isfinite(flux)):
            raise ValueError("non-finite model flux")
        return flux
    except Exception:
        # Return large residuals instead of crashing optimizer
        return np.full_like(t, 1e12, dtype=float)

popt, pcov = curve_fit(
    model_flux,
    x_fit,
    y_fit,
    p0=p0,
    sigma=sigma_fit,
    absolute_sigma=True,
    bounds=(lb, ub),
    method='trf',
    maxfev=50000,
    check_finite=False,
    nan_policy='omit',
)

end_time = time.time()
print(f"1S1L curve-fit completed in {(end_time - start_time):.2f} seconds")

print("\nResults:")
cf_linear_sol = {}
for i, param in enumerate(parameters_to_fit_1s1l):
    err_i = np.sqrt(pcov[i, i]) if np.isfinite(pcov[i, i]) and pcov[i, i] >= 0 else np.nan
    print(f"  {param}: {popt[i]:.6f} ± {err_i:.6f}")

print("\nComparison with truth:")
for i, param in enumerate(parameters_to_fit_1s1l):
    diff = popt[i] - truths_transformed[i]
    err_i = np.sqrt(pcov[i, i]) if np.isfinite(pcov[i, i]) and pcov[i, i] >= 0 else np.nan
    sigma_txt = f"{abs(diff)/err_i:.2f}σ" if np.isfinite(err_i) and err_i > 0 else "n/a"
    print(f"  {param}: {diff:.6f} ({sigma_txt})")
    # Convert best fit parameters back to physical units for plotting
    if 'log' in param:
        physical_value = 10**popt[i]
        label = param.replace('log_', '').replace('log', '')
    else:
        physical_value = popt[i]
        label = param
    cf_linear_sol[label] = physical_value

Using these simpler methods for binary-lens fits can be a terrible idea, because the chi-squared surface is very complex and full of local minima; MCMC is a safer choice for binary lens fits.

In [ ]:
#@title Plot 1S1L results (MCMC vs Nelder-Mead vs curve_fit)
# Build physical-parameter dictionaries for each method
mcmc_linear_sol = {}
for i, p in enumerate(parameters_to_fit_1s1l):
    key = p.replace('log_', '').replace('log', '') if 'log' in p else p
    val = 10.0**best_fit_1s1l[i] if 'log' in p else best_fit_1s1l[i]
    mcmc_linear_sol[key] = val

# nm_linear_sol and cf_linear_sol are created in previous cells
print('MCMC:', mcmc_linear_sol)
print('Nelder-Mead:', nm_linear_sol)
print('curve_fit:', cf_linear_sol)

def apply_solution_to_model(event, sol_dict):
    """Apply a solution dictionary to MulensModel parameters with simple alias handling."""
    alias = {
        'tE': 't_E',
        'u0': 'u_0',
        't0': 't_0',
        'piE_E': 'pi_E_E',
        'piE_N': 'pi_E_N',
    }
    for k, v in sol_dict.items():
        pname = alias.get(k, k)
        if hasattr(event.model.parameters, pname):
            setattr(event.model.parameters, pname, float(v))
    event.fit_fluxes()

def plot_1s1l_model_comparison(event, xlim=None):
    plt.figure(figsize=(12, 6))
    event.plot_data(zorder=0)

    # MCMC
    apply_solution_to_model(event, mcmc_linear_sol)
    event.plot_model(color='red', label='MCMC best fit', zorder=2)

    # Nelder-Mead
    apply_solution_to_model(event, nm_linear_sol)
    event.plot_model(color='royalblue', linestyle='--', label='Nelder-Mead', zorder=3)

    # curve_fit
    apply_solution_to_model(event, cf_linear_sol)
    event.plot_model(color='seagreen', linestyle=':', linewidth=2, label='curve_fit', zorder=4)

    plt.title('1S1L Synthetic Event - Model Comparison')
    plt.xlabel('HJD')
    plt.ylabel('Magnitude')
    if xlim is not None:
        plt.xlim(*xlim)
    plt.legend()
    plt.show()

def solution_to_fit_vector(sol_dict, param_names):
    """Convert physical-parameter solution dict into the parameterization used in corner plot."""
    alias = {
        'tE': 't_E',
        'u0': 'u_0',
        't0': 't_0',
        'piE_E': 'pi_E_E',
        'piE_N': 'pi_E_N',
    }
    inv_alias = {v: k for k, v in alias.items()}
    vec = []
    for p in param_names:
        base = p.replace('log_', '').replace('log', '') if 'log' in p else p
        candidates = [base, inv_alias.get(base, None)]
        value = None
        for c in candidates:
            if c is not None and c in sol_dict:
                value = sol_dict[c]
                break
        if value is None:
            vec.append(np.nan)
            continue
        vec.append(np.log10(value) if 'log' in p else value)
    return np.array(vec, dtype=float)

# Full range
plot_1s1l_model_comparison(event_1s1l)

# Zoomed range
plot_1s1l_model_comparison(event_1s1l, xlim=(2459810.0, 2459822.0))

# Corner plot (MCMC samples) + annotate best-fit points from all 3 methods
fig = corner.corner(
    samples_1s1l,
    labels=parameters_to_fit_1s1l,
    truths=truths_transformed,
    quantiles=[0.16, 0.5, 0.84],
)

mcmc_vec = solution_to_fit_vector(mcmc_linear_sol, parameters_to_fit_1s1l)
nm_vec = solution_to_fit_vector(nm_linear_sol, parameters_to_fit_1s1l)
cf_vec = solution_to_fit_vector(cf_linear_sol, parameters_to_fit_1s1l)

if np.all(np.isfinite(mcmc_vec)):
    corner.overplot_lines(fig, mcmc_vec, color='red', linewidth=1.5)
    corner.overplot_points(fig, mcmc_vec[None, :], color='red', marker='o', ms=4)
if np.all(np.isfinite(nm_vec)):
    corner.overplot_lines(fig, nm_vec, color='royalblue', linewidth=1.5)
    corner.overplot_points(fig, nm_vec[None, :], color='royalblue', marker='s', ms=4)
if np.all(np.isfinite(cf_vec)):
    corner.overplot_lines(fig, cf_vec, color='seagreen', linewidth=1.5)
    corner.overplot_points(fig, cf_vec[None, :], color='seagreen', marker='^', ms=4)

legend_handles = [
    plt.Line2D([0], [0], color='red', marker='o', lw=1.5, label='MCMC'),
    plt.Line2D([0], [0], color='royalblue', marker='s', lw=1.5, label='Nelder-Mead'),
    plt.Line2D([0], [0], color='seagreen', marker='^', lw=1.5, label='curve_fit'),
]
fig.axes[0].legend(handles=legend_handles, loc='upper right', frameon=True)

plt.suptitle('1S1L Event - Parameter Posteriors', y=1.02)
plt.show()

# Plot updated caustic structure using MCMC best fit
apply_solution_to_model(event_1s1l, mcmc_linear_sol)
plt.figure(figsize=(8, 8))
event_1s1l.model.plot_caustics()
event_1s1l.model.plot_trajectory()
plt.title('1S1L Event - Best Fit Caustic Structure (MCMC)')
plt.xlabel('x (Einstein radii)')
plt.ylabel('y (Einstein radii)')
plt.axis('equal')
plt.show()

#### 1.5.2 Single Source Binary Lens (1S2L)

Now let's demonstrate a binary lens model. We'll use a simulated binary lens event to show how to fit for the additional parameters; `s`, `q`, and `alpha`.We will continue to include finite source and parallax effects.

In [ ]:
#@title Load metadata for 1S2L example
# For this example, we'll use a simulated binary lens event
event_idx_bl = 5  # Binary lens event
data_file_bl = merged_bl_df['lc_file_path_W149'][event_idx_bl]
ra_bl = merged_bl_df['RA_(deg)'][event_idx_bl]
dec_bl = merged_bl_df['Dec_(deg)'][event_idx_bl]
gamma_bl = merged_bl_df['gamma'][event_idx_bl]
coord_bl = SkyCoord(ra=ra_bl * u.deg, dec=dec_bl * u.deg, frame='icrs')

# Get truth parameters for comparison
# NOTE: convert challenge-relative t0 to HJD to match the time stamps in the light-curve files
t_0_truth_bl = float(merged_bl_df['t0'][event_idx_bl]) + 2458233.5
u_0_truth_bl = float(merged_bl_df['u0'][event_idx_bl])
t_E_truth_bl = float(merged_bl_df['tE'][event_idx_bl])
rho_truth_bl = float(merged_bl_df['rhos'][event_idx_bl])
piE_truth_bl = float(merged_bl_df['piE'][event_idx_bl])
s_truth_bl = float(merged_bl_df['s'][event_idx_bl])
q_truth_bl = float(merged_bl_df['q'][event_idx_bl])
alpha_truth_bl = float(merged_bl_df['alpha'][event_idx_bl])

parameters_to_fit_1s2l = ["t_0", "u_0", "logt_E", "logrho", "pi_E_N", "pi_E_E", "logs", "logq", "alpha"]
truths_transformed_1s2l = [
    t_0_truth_bl,
    u_0_truth_bl,
    np.log10(t_E_truth_bl),
    np.log10(rho_truth_bl),
    np.sqrt(piE_truth_bl**2/2.0),
    np.sqrt(piE_truth_bl**2/2.0),
    np.log10(s_truth_bl),
    np.log10(q_truth_bl),
    alpha_truth_bl
]

print(f"Event {event_idx_bl}: Truth parameters")
print(f"  t_0: {t_0_truth_bl:.3f}")
print(f"  u_0: {u_0_truth_bl:.4f}")
print(f"  t_E: {t_E_truth_bl:.2f} days")
print(f"  ρ: {rho_truth_bl:.6f}")
print(f"  π_E: {piE_truth_bl:.2f} mas")
print(f"  s: {s_truth_bl:.3f}")
print(f"  q: {q_truth_bl:.3e}")
print(f"  α: {alpha_truth_bl:.3f}")

# Create initial model with finite source effects
initial_params_1s2l = {
    't_0': t_0_truth_bl, # this is in essence already perturbed because the simulations define t_0 as time of closest approach to the lens host
    'u_0': u_0_truth_bl * -1.0,  # same for u_0. And I guess Mulensmodel and the simulations define u_0 as flipped from each other
    't_E': t_E_truth_bl * 1.05,
    'rho': rho_truth_bl * 0.95,
    'pi_E_E': np.sqrt(piE_truth_bl**2/2.0),
    'pi_E_N': np.sqrt(piE_truth_bl**2/2.0),
    's': s_truth_bl * 1.05,
    'q': q_truth_bl * 1.05,
    'alpha': alpha_truth_bl * 1.05,
    't_0_par': t_0_truth_bl
}
log_initial_params_1s2l = {
    't_0': initial_params_1s2l['t_0'],  
    'u_0': initial_params_1s2l['u_0'],  
    'logt_E': np.log10(initial_params_1s2l['t_E']), 
    'logrho': np.log10(initial_params_1s2l['rho']),
    'pi_E_E': initial_params_1s2l['pi_E_E'],  
    'pi_E_N': initial_params_1s2l['pi_E_N'],  
    'logs': np.log10(initial_params_1s2l['s']),
    'logq': np.log10(initial_params_1s2l['q']),
    'alpha': initial_params_1s2l['alpha']
}
initial_guess_1s2l = list(log_initial_params_1s2l.values())
print(log_initial_params_1s2l)

NameError: name 'merged_bl_df' is not defined

In [ ]:
#@title Create 1S2L data object
# Optional controls
FIT_WINDOW_IN_TE = 3.0      # fit only data within t0 ± FIT_WINDOW_IN_TE * tE (it's handy to have a good guess for tE)
USE_CROPPED_DATA = True      # set False to use full light curve (slower)

# Load the light curve data
data_1s2l = mm.MulensData(file_name=data_file_bl,
                          phot_fmt='mag',
                          ephemerides_file=EPHEM_FILE,
                          plot_properties={'color': '#a859e4', 'label': 'Roman W149'},
                          bandpass='H')

# Robust magnification-method setup for far-wing epochs
tE_bl = max(t_E_truth_bl, 1e-6)

if USE_CROPPED_DATA:
    t1_bl = t_0_truth_bl - FIT_WINDOW_IN_TE * tE_bl
    t2_bl = t_0_truth_bl + FIT_WINDOW_IN_TE * tE_bl
else:
    t1_bl = np.min(data_1s2l.time) - 5.0
    t2_bl = np.max(data_1s2l.time) + 5.0

# Optional time-window crop for speed
if USE_CROPPED_DATA:
    m = (data_1s2l.time >= t1_bl) & (data_1s2l.time <= t2_bl)
    # replace the data object
    data_1s2l = mm.MulensData(
        data_list=[data_1s2l.time[m], data_1s2l.mag[m], data_1s2l.err_mag[m]],
        phot_fmt='mag',
        ephemerides_file=EPHEM_FILE,
        plot_properties={'color': '#a859e4', 'label': 'Roman W149 (cropped)'},
        bandpass='H',
    )
    print(f'Using cropped data: {m.sum()} points in [{t_min:.2f}, {t_max:.2f}]')
else:
    print(f'Using full data: {len(data_1s1l.time)} points')

In [ ]:
#@title Create 1S2L model object
# Create initial model (slightly perturbed from truth)
model_1s2l = mm.Model(initial_params_1s2l, coords=coord_bl, ephemerides_file=EPHEM_FILE)

# In older MulensModel versions, 'VBBL' is recognized; in newer docs you may see 'VBM'.
model_1s2l.set_magnification_methods([
    t1_bl, 'VBBL', t2_bl
])
model_1s2l.set_limb_coeff_gamma('H', float(gamma_bl))  # resolves H-band limb-darkening warning

event_1s2l = mm.Event(datasets=data_1s2l, model=model_1s2l)
event_1s2l.fit_fluxes()

# Plot the initial model
plt.figure(figsize=(12, 6))
event_1s2l.plot_data(zorder=0)
# IMPORTANT: set_magnification_methods() does NOT change plot length;
# it only changes which solver is used at different epochs.
# To draw a longer model, evaluate on a custom time grid explicitly.
([F_S], F_B) = event_1s2l.get_flux_for_dataset(event_1s2l.datasets[0])
t_plot = np.linspace(t1_bl, t2_bl, 2000)
flux_model = model_1s2l.get_magnification(t_plot) * F_S + F_B
mag_model = mm.Utils.get_mag_from_flux(flux_model)
plt.plot(t_plot, mag_model, color='black', alpha=0.8, lw=1.2, label='Initial Model (extended)', zorder=2)
plt.title("1S2L Synthetic Event - Initial Model")
plt.xlabel("HJD")
plt.ylabel("Magnitude")
plt.legend()
plt.show()

# Plot the initial model
plt.figure(figsize=(12, 6))
event_1s2l.plot_data(zorder=0)
plt.plot(t_plot, mag_model, color='black', alpha=0.8, lw=1.2, label='Initial Model (extended)', zorder=2)
plt.title("1S2L Synthetic Event - Initial Model")
plt.xlim(t1_bl, t2_bl)
plt.xlabel("HJD")
plt.ylabel("Magnitude")
plt.legend()
plt.show()

# Plot caustic structure
plt.figure(figsize=(8, 8))
event_1s2l.model.plot_caustics()
event_1s2l.model.plot_trajectory()
plt.title("1S2L Event - Caustic Structure")
plt.xlabel("x (Einstein radii)")
plt.ylabel("y (Einstein radii)")
plt.axis('equal')
plt.show()

We are going to cheat a little here and simply run `emcee` starting from our purturbed truths. In practice, outside of simulated examples where we know the true parameters, choosing a starting point for binary-lens, MCMC runs is far from trivial and strategies for this process are the subject of our [Binary Lens Fitting](c_binary_lens.ipynb) notebook ([external Link](https://github.com/rges-pit/data-challenge-notebooks/blob/main/AAS%20Workshop/Session%20C%3A%20Binary%20Lens/Fitting_Binary_Lenses.ipynb)).

In [ ]:
#@title Fit 1S2L model with MCMC
# Set up parameters to fit (excluding rho for simplicity in this example)
print("Starting 1S2L MCMC fit...")
start_time = time.time()

# Run MCMC with more walkers and steps for binary lens 
# It takes longer to explore larger parameter space (models with more parameters)
#sampler_1s2l = run_mcmc_fit(event_1s2l, parameters_to_fit_1s2l, initial_guess_1s2l,
#                           nwalkers=32, nsteps=500, burnin=250)  # takes ~2hrs on the Large (CPU Optimized) Nexus server
#sampler_1s2l = run_mcmc_fit(event_1s2l, parameters_to_fit_1s2l, initial_guess_1s2l,
#                           nwalkers=200, nsteps=5000, burnin=4000)  # probably more reasonable for nearby convergence and 
                                                                   # a well sampled posterior.
sampler_1s2l = run_mcmc_fit(event_1s2l, parameters_to_fit_1s2l, initial_guess_1s2l,
                           nwalkers=32, nsteps=300, burnin=100)  # isn't going to converge, but might actually finish in a 
                                                                 # reasonable time

# Get results
samples_1s2l = sampler_1s2l.chain[:, 200:, :].reshape((-1, 6))
best_fit_1s2l = np.median(samples_1s2l, axis=0)
uncertainties_1s2l = np.std(samples_1s2l, axis=0)

fit_time = time.time() - start_time
print(f"\n1S2L Fit completed in {fit_time:.1f} seconds")
print("\nResults:")
for i, param in enumerate(parameters_to_fit_1s2l):
    print(f"  {param}: {best_fit_1s2l[i]:.6f} ± {uncertainties_1s2l[i]:.6f}")

# Compare with truth
print("\nComparison with truth:")
truth_values_1s2l = [truths_transformed_1s2l[i] for i, p in enumerate(parameters_to_fit_1s2l)]
for i, param in enumerate(parameters_to_fit_1s2l):
    diff = best_fit_1s2l[i] - truth_values_1s2l[i]
    sigma_diff = abs(diff) / uncertainties_1s2l[i] if uncertainties_1s2l[i] > 0 else 0
    print(f"  {param}: {diff:.6f} ({sigma_diff:.2f}σ)")

In [ ]:
#@title Plot 1S2L results
# Update model with best fit parameters
for i, param_name in enumerate(parameters_to_fit_1s2l):
    if 'log' in param_name:
        physical_value = 10.0**best_fit_1s2l[i]
    else:
        physical_value = best_fit_1s2l[i]
    setattr(event_1s2l.model.parameters, param_name, physical_value)
event_1s2l.fit_fluxes()

# Plot light curve with best fit
plt.figure(figsize=(12, 6))
event_1s2l.plot_data()
event_1s2l.plot_model(color='red', label='Best Fit')
plt.gca().invert_yaxis()
plt.title("1S2L Synthetic Event - Best Fit Model")
plt.xlabel("HJD")
plt.ylabel("Magnitude")
plt.legend()
plt.show()

# Plot corner plot
fig = corner.corner(samples_1s2l, labels=parameters_to_fit_1s2l,
                    truths=truth_values_1s2l, quantiles=[0.16, 0.5, 0.84])
plt.suptitle("1S2L Event - Parameter Posteriors", y=1.02)
plt.show()

# Plot updated caustic structure
plt.figure(figsize=(8, 8))
event_1s2l.model.plot_caustics()
plt.title("1S2L Event - Best Fit Caustic Structure")
plt.xlabel("x (Einstein radii)")
plt.ylabel("y (Einstein radii)")
plt.axis('equal')
plt.show()

#### 1.5.3 Single Source Triple Lens (1S3L)

Triple lens systems are rare but important for detecting hierarchical planetary systems. Here we demonstrate how to set up an example triple lens model.

**Triple Lens Model Parameters:**
- **Primary lens**: t_0, u_0, t_E
- **First companion**: q1, s1, alpha1
- **Second companion**: q2, s2, alpha2, phi
- **Finite source**: rho
- **Higher order**: pi_E_N, pi_E_E

**Example Triple Lens Parameters (hierarchical system):**
- t_0: 2459123.5
- u_0: 0.1
- t_E: 25.0
- q1: 0.1 (First companion - planet)
- s1: 1.2 (Separation of first companion)
- alpha1: 45.0
- q2: 0.01 (Second companion - moon)
- s2: 0.1 (Separation of second companion relative to first)
- alpha2: 30.0
- phi: 60.0 (Angle between companions)
- rho: 0.001 (Finite source)

> **Note**: This represents a hierarchical triple system with:
> - Primary lens (star)
> - First companion (planet at ~1.2 Einstein radii)
> - Second companion (moon at ~0.1 Einstein radii from planet)

**MulensModel doesn't natively support triple lenses** (as of version 3.9.0), but here's how you could extend it:

**Triple Lens Implementation Options:**

**1. Use VBMicrolensing:**
- Native support for multiple lenses
- More efficient for complex lens systems
- Better suited for hierarchical systems

**2. Extend MulensModel:**
- Create custom model class
- Implement triple lens magnification calculation
- Requires significant development effort

**3. Use triplelens package:**
- Specialized for triple lens calculations
- GitHub: https://github.com/rkkuang/triplelens
- Limited integration with other tools

**4. Approximate approach:**
- Treat as hierarchical binary systems
- Fit inner binary first, then outer binary
- May miss some triple lens effects

**Framework for Triple Lens Fitting:**
```python
parameters_to_fit_1s3l = [
    't_0', 'u_0', 't_E',
    'q1', 's1', 'alpha1',
    'q2', 's2', 'alpha2', 'phi',
    'rho'
]
```

#### <font face="Helvetica" size="4"> 1.5.4 Binary Source Single Lens (2S1L) </font>

Binary source events occur when the source star is actually a binary system.  This can create distinctive light-curve features, but it can also look remarkably like some binary-lens events. We don't have any example binary-source data, so we will have to simulate some.

> At least one of the sources **must** be luminous and there is no need to model a non-luminous source if Xalarap is not a significant effect.

In [ ]:
#@title 2S1L Model Setup
# Binary source parameters:
# - t_0, u_0, t_E: standard microlensing parameters
# - q_S: flux ratio of source components
# - rho_S: angular separation of source components
# - theta_S: angle of source binary

# Create synthetic binary source data
def create_binary_source_data(t_0=2459123.5, u_0=0.1, t_E=25.0, q_S=0.8, rho_S=0.01, theta_S=45.0):
    """Create synthetic binary source light curve"""
    # Time array around the event
    t = np.linspace(t_0 - 2*t_E, t_0 + 2*t_E, 200)

    # Create binary source model
    params = {
        't_0': t_0,
        'u_0': u_0,
        't_E': t_E,
        'q_S': q_S,      # flux ratio
        'rho_S': rho_S,  # angular separation
        'theta_S': theta_S  # angle
    }

    # Note: MulensModel doesn't natively support binary sources
    # This is a simplified approximation
    model = mm.Model({'t_0': t_0, 'u_0': u_0, 't_E': t_E},
                     coords=coord, ephemerides_file=EPHEM_FILE)

    # Calculate magnification for single source
    magnification = model.get_magnification(t)

    # Approximate binary source effect
    # This is a simplified treatment - real binary sources are more complex
    mag_0 = 18.0
    mag = mag_0 - 2.5 * np.log10(magnification)

    # Add some binary source features (simplified)
    # In reality, this would depend on the source binary orientation
    binary_modulation = 0.1 * np.sin(2 * np.pi * (t - t_0) / t_E)
    mag += binary_modulation

    mag_err = 0.01 * np.ones_like(mag)
    mag += np.random.normal(0, mag_err)

    return t, mag, mag_err, params

# Create the synthetic data
t_binary_source, mag_binary_source, mag_err_binary_source, truth_params_binary_source = create_binary_source_data()

print("Synthetic 2S1L Event - Truth Parameters:")
for key, value in truth_params_binary_source.items():
    print(f"  {key}: {value}")

print("\nNote: This is a simplified binary source model.")
print("Real binary source events require more sophisticated modeling.")

In [ ]:
#@title Create 2S1L data object
# Create data object from synthetic data
data_list_2s1l = [t_binary_source, mag_binary_source, mag_err_binary_source]
data_2s1l = mm.MulensData(data_list=data_list_2s1l,
                          phot_fmt='mag',
                          ephemerides_file=EPHEM_FILE,
                          plot_properties={'color': '#a859e4', 'label': 'Synthetic Binary Source'},
                          bandpass='H')

# Create initial model (treating as single source for now)
params_2s1l = {
    't_0': truth_params_binary_source['t_0'],
    'u_0': truth_params_binary_source['u_0'],
    't_E': truth_params_binary_source['t_E']
}
model_2s1l = mm.Model(params_2s1l, coords=coord, ephemerides_file=EPHEM_FILE)
event_2s1l = mm.Event(datasets=data_2s1l, model=model_2s1l)

# Plot the data
plt.figure(figsize=(12, 6))
event_2s1l.plot_data()
event_2s1l.plot_model(color='red', label='Single Source Model')
plt.gca().invert_yaxis()
plt.title("2S1L Synthetic Event - Data with Single Source Model")
plt.xlabel("HJD")
plt.ylabel("Magnitude")
plt.legend()
plt.show()

print("Note: The residuals show the binary source signature.")
print("A proper binary source model would fit these features.")

### 1.6 Higher-Order Effects Summary

We've demonstrated several key higher-order effects in microlensing, throughout this `MulensModel` section, including:

**Finite Source Effects (ρ):**
- Important when the source star is resolved
- Provides constraints on the lens mass and distance
- Essential for accurate parameter estimation

**Parallax Effects (π_E):**
- Crucial for space-based observations
- Enables mass and distance measurements
- Breaks degeneracies in lens properties

**Other Effects (not demonstrated):**
- **Lens Orbital Motion:** Changes in binary lens separation over time (often highly degenerate with parllax effect)
- **Xallarap:** Source orbital motion effects
- **Limb Darkening:** Non-uniform source brightness (rarely well constrained). We have used `gamma` (linear limb-darkening) in this section, but it has always been set to the simulation truth, 0.0; i.e., uniform radial source brightness.
- **Astrometry:** Position measurements during events (sparsley obtained prior to Roman). Not yet supported in `MulensModel`.

**Modelling Best Practices:**
- Start with simple models and add complexity gradually
- Use MCMC for complex parameter spaces
- Consider known degeneracies
- Consider computational efficiency for large surveys
- If you would like to request new features in `MulensModel`, kindly [open an issue on GitHub](https://github.com/rpoleski/MulensModel/issues/new).

## 2. pyLIMA

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;">

pyLIMA (Python Lightcurve Interpretation and Microlensing Analysis) is the first open-source software for modeling microlensing events. It was developed by Etienne Bachelet and provides a flexible, modular framework for microlensing analysis. pyLIMA is particularly well-suited for space-based observations and offers advanced features like simulation capabilities and custom parameter definitions.

**Key Features:**
- Modular design with separate event, telescope, and model classes
- Multiple fitting algorithms (LM, DE, MCMC, TRF)
- Built-in simulation capabilities
- Custom parameter definitions
- Support for multiple telescopes and filters
- Limb darkening and finite source effects

### <font face="Helvetica" size="5"> 2.1 Installation and Setup </font>

pyLIMA can be installed via pip or conda. It has several dependencies including numpy, scipy, matplotlib, and astropy.

In [ ]:
#@title Install pyLIMA
# Import pyLIMA modules
from pyLIMA import event, telescopes
from pyLIMA.models import PSPL_model, FSPL_model
from pyLIMA.fits import LM_fit, DE_fit, MCMC_fit, TRF_fit
from pyLIMA.outputs import pyLIMA_plots
from pyLIMA.simulations import simulator

print("pyLIMA installed and imported successfully!")

### <font face="Helvetica" size="5"> 2.2 Basic Usage </font>

pyLIMA uses a modular approach with three main components:
1. **Event**: Contains all information about the microlensing event
2. **Telescopes**: Individual telescope data and properties
3. **Models**: The microlensing model to fit

In [ ]:
#@title Basic pyLIMA Setup
# Create a new event
Roman_event = event.Event()
Roman_event.name = f'pyLIMA_1s1l_{event_idx}'

# Create synthetic data for demonstration
# In practice, you would load real data from files
np.random.seed(42)  # For reproducible results

# Create telescope object
lightcurve_data = np.column_stack(
    [
        event_1s1l.datasets[0].time, 
        event_1s1l.datasets[0].mag, 
        event_1s1l.datasets[0].err_mag
        ]
    )
Roman_W149 = telescopes.Telescope(
    name='Roman_W149',
    camera_filter='H',
    lightcurve=lightcurve_data.astype(float),
    lightcurve_names=['time', 'mag', 'err_mag'],
    lightcurve_units=['BJD', 'mag', 'mag']
)

# Add telescope to event
Roman_event.telescopes.append(Roman_W149)

# Set survey telescope (for alignment)
Roman_event.find_survey('OGLE')

# Check event setup
Roman_event.check_event()

print(f"Event '{Roman_event.name}' created with {len(Roman_event.telescopes)} telescope(s)")
print(f"Data points: {len(Roman_W149.lightcurve)}")

### <font face="Helvetica" size="5"> 2.3 Model Examples </font>

pyLIMA supports various microlensing models. Let's start with the basic Point Source Point Lens (PSPL) model.

#### <font face="Helvetica" size="4"> 2.3.1 Single Lens (PSPL) Example </font>

The PSPL model is the simplest microlensing model with parameters: `t_0`, `u_0`, `t_E`, and flux parameters for each telescope.

In [ ]:
#@title PSPL Model Setup and Fitting
# Create PSPL model
pspl = PSPL_model.PSPLmodel(your_event)

# Initialize fit with Levenberg-Marquardt algorithm
my_fit = LM_fit.LMfit(pspl)

# Show initial parameters
print("Initial fit parameters:")
print(my_fit.fit_parameters)

# Run the fit
print("\nRunning LM fit...")
my_fit.fit()
my_fit.fit_outputs()

# Display results
print("\nFit Results:")
print(my_fit.fit_results['best_model'])
print(f"\nChi-squared: {my_fit.fit_results['chi2']:.2f}")

# Plot the results
pyLIMA_plots.plot_lightcurves(pspl, my_fit.fit_results['best_model'])
plt.title("pyLIMA PSPL Fit")
plt.show()

In [ ]:
#@title PSPL with MCMC
# Use MCMC for better parameter estimation
mcmc_fit = MCMC_fit.MCMCfit(pspl, rescale_photometry=True)

# Use LM results as initial guess
mcmc_fit.model_parameters_guess = my_fit.fit_results['best_model'][:3]  # t0, u0, tE

print("Running MCMC fit (this may take a while)...")
mcmc_fit.fit()
mcmc_fit.fit_outputs()

# Get MCMC results
MCMC_results = mcmc_fit.fit_results['MCMC_chains']
burnin = 1000

# Calculate statistics
posterior_samples = MCMC_results[burnin:, :, :3]  # t0, u0, tE
best_fit_mcmc = np.median(posterior_samples, axis=(0, 1))
uncertainties_mcmc = np.std(posterior_samples, axis=(0, 1))

print("\nMCMC Results:")
param_names = ['t_0', 'u_0', 't_E']
for i, name in enumerate(param_names):
    print(f"{name}: {best_fit_mcmc[i]:.4f} ± {uncertainties_mcmc[i]:.4f}")

# Compare with truth values
print("\nComparison with truth:")
truth_values = [t0, u0, tE]
for i, name in enumerate(param_names):
    diff = best_fit_mcmc[i] - truth_values[i]
    sigma_diff = abs(diff) / uncertainties_mcmc[i]
    print(f"{name}: {diff:.4f} ({sigma_diff:.2f}σ)")

#### <font face="Helvetica" size="4"> 2.3.2 Finite Source (FSPL) Example </font>

The FSPL model includes finite source effects with the ρ parameter, which is crucial for high-magnification events.

In [ ]:
#@title FSPL Model with Finite Source Effects
# Create FSPL model
fspl = FSPL_model.FSPLmodel(your_event)

# Set limb darkening coefficient
your_event.telescopes[0].ld_gamma = 0.5

# Use differential evolution for better exploration
de_fit = DE_fit.DEfit(fspl, loss_function='chi2')

print("Running DE fit for FSPL model...")
de_fit.fit()

# Display results
print("\nFSPL Fit Results:")
print(de_fit.fit_results['best_model'])
print(f"\nChi-squared: {de_fit.fit_results['chi2']:.2f}")

# Plot results
pyLIMA_plots.plot_lightcurves(fspl, de_fit.fit_results['best_model'])
plt.title("pyLIMA FSPL Fit with Finite Source Effects")
plt.show()

# Compare PSPL vs FSPL
print("\nModel Comparison:")
print(f"PSPL χ²: {my_fit.fit_results['chi2']:.2f}")
print(f"FSPL χ²: {de_fit.fit_results['chi2']:.2f}")
print(f"Improvement: {my_fit.fit_results['chi2'] - de_fit.fit_results['chi2']:.2f}")

#### <font face="Helvetica" size="4"> 2.3.3 Simulation Capabilities </font>

One of pyLIMA's unique features is its built-in simulation capabilities, allowing you to generate realistic microlensing light curves.

In [ ]:
#@title pyLIMA Simulation Example
# Create a new event for simulation
sim_event = event.Event(ra=270, dec=-30)
sim_event.name = 'Simulated Event'

# Create a realistic telescope with observing constraints
CTIO_I = simulator.simulate_a_telescope(
    name='CTIO_I',
    time_start=2457365.5,
    time_end=2457965.5,
    sampling=4,
    location='Earth',
    camera_filter='I',
    uniform_sampling=False,
    altitude=1000,
    longitude=-109.285399,
    latitude=-27.130,
    bad_weather_percentage=10.0/100,
    moon_windows_avoidance=30,
    minimum_alt=30,
    astrometry=False
)

sim_event.telescopes.append(CTIO_I)
sim_event.check_event()

print(f"Simulated telescope '{CTIO_I.name}' created")
print(f"Observation period: {CTIO_I.time_start:.1f} to {CTIO_I.time_end:.1f}")
print(f"Sampling: {CTIO_I.sampling} hours")

In [ ]:
#@title Generate Simulated Light Curve
# Create PSPL model for simulation
sim_pspl = PSPL_model.PSPLmodel(sim_event)

# Generate random model parameters
sim_parameters = simulator.simulate_microlensing_model_parameters(sim_pspl)
print("Simulated Parameters:")
print(sim_parameters)

# Convert to pyLIMA format
pyLIMA_parameters = sim_pspl.compute_pyLIMA_parameters(sim_parameters)

# Generate the light curve
simulator.simulate_lightcurve(sim_pspl, pyLIMA_parameters)

# Plot the simulated light curve
pyLIMA_plots.plot_lightcurves(sim_pspl, sim_parameters)
plt.title("pyLIMA Simulated Light Curve")
plt.show()

print(f"Generated {len(CTIO_I.lightcurve)} data points")
print(f"Time range: {CTIO_I.lightcurve['time'].min():.1f} to {CTIO_I.lightcurve['time'].max():.1f}")

In [ ]:
#@title Fit Simulated Data
# Now fit the simulated data to recover parameters
sim_fit = LM_fit.LMfit(sim_pspl)
sim_fit.fit()
sim_fit.fit_outputs()

print("Fit Results vs Truth:")
print("Parameter | Truth    | Fit      | Difference")
print("---------|----------|----------|-----------")

param_names = list(sim_pspl.model_dictionnary.keys())
for i, name in enumerate(param_names[:3]):  # t0, u0, tE
    truth = sim_parameters[i]
    fit = sim_fit.fit_results['best_model'][i]
    diff = fit - truth
    print(f"{name:8} | {truth:8.4f} | {fit:8.4f} | {diff:+8.4f}")

print(f"\nChi-squared: {sim_fit.fit_results['chi2']:.2f}")

# Plot fit results
pyLIMA_plots.plot_lightcurves(sim_pspl, sim_fit.fit_results['best_model'])
plt.title("Fitting Simulated Data")
plt.show()

### <font face="Helvetica" size="5"> 2.4 Advanced Features </font>

pyLIMA offers several advanced features including custom parameter definitions and multiple fitting algorithms.

#### <font face="Helvetica" size="4"> 2.4.1 Custom Parameter Definitions </font>

pyLIMA allows you to define custom parameter transformations, which can be useful for better sampling or physical interpretation.

In [ ]:
#@title Custom Parameter Definitions
# Define custom parameters: log_tE instead of tE
class MyFancyParameters(object):
    def __init__(self, fancy_parameters={'tE': 'log_tE'},
                 fancy_boundaries={'log_tE': (0, 3)}):
        self.fancy_parameters = fancy_parameters
        self.fancy_boundaries = fancy_boundaries

    def tE(self, fancy_params):
        return 10**fancy_params['log_tE']

    def log_tE(self, standard_params):
        return np.log10(standard_params['tE'])

# Create model with custom parameters
my_pars = MyFancyParameters()
fspl_custom = FSPL_model.FSPLmodel(your_event, fancy_parameters=my_pars)

print("Custom parameter model created")
print(f"Parameters: {fspl_custom.fit_parameters.keys()}")
print(f"Custom boundaries: {my_pars.fancy_boundaries}")

In [ ]:
#@title Fit with Custom Parameters
# Use Trust Region Reflective algorithm
trf_fit = TRF_fit.TRFfit(fspl_custom)

# Set initial guess (convert tE to log_tE)
guess_parameters = [t0, u0, np.log10(tE), 0.001]  # t0, u0, log_tE, rho
trf_fit.model_parameters_guess = guess_parameters

print("Running TRF fit with custom parameters...")
trf_fit.fit()

print("\nFit Results (custom parameters):")
print(trf_fit.fit_results['best_model'])
print(f"\nChi-squared: {trf_fit.fit_results['chi2']:.2f}")

# Convert back to standard parameters for comparison
custom_params = trf_fit.fit_results['best_model']
standard_tE = 10**custom_params[2]  # Convert log_tE back to tE
print(f"\nConverted t_E: {standard_tE:.2f} days")
print(f"Truth t_E: {tE:.2f} days")
print(f"Difference: {standard_tE - tE:.2f} days")

#### <font face="Helvetica" size="4"> 2.4.2 Multiple Telescopes and Filters </font>

pyLIMA excels at handling data from multiple telescopes and filters, which is common in modern microlensing surveys.

In [ ]:
#@title Multiple Telescope Example
# Create a new event with multiple telescopes
multi_event = event.Event()
multi_event.name = 'Multi-Telescope Event'

# Generate data for multiple telescopes with different properties
telescopes_data = []
telescope_names = ['OGLE', 'MOA', 'LCO']
filters = ['I', 'R', 'V']
offsets = [0.0, 0.1, -0.05]  # Different zero points
noise_levels = [0.01, 0.015, 0.02]  # Different noise levels

for i, (name, filt, offset, noise) in enumerate(zip(telescope_names, filters, offsets, noise_levels)):
    # Generate light curve with different properties
    mag_tel = mag + offset + np.random.normal(0, noise, len(mag))
    mag_err_tel = noise * np.ones_like(mag)

    lightcurve_data = np.column_stack([times, mag_tel, mag_err_tel])

    telescope = telescopes.Telescope(
        name=name,
        camera_filter=filt,
        lightcurve=lightcurve_data.astype(float),
        lightcurve_names=['time', 'mag', 'err_mag'],
        lightcurve_units=['JD', 'mag', 'mag']
    )

    multi_event.telescopes.append(telescope)
    telescopes_data.append(lightcurve_data)

# Set survey telescope
multi_event.find_survey('OGLE')
multi_event.check_event()

print(f"Created event with {len(multi_event.telescopes)} telescopes:")
for tel in multi_event.telescopes:
    print(f"  {tel.name} ({tel.camera_filter}-band): {len(tel.lightcurve)} points")

In [ ]:
#@title Fit Multi-Telescope Data
# Create PSPL model for multi-telescope data
multi_pspl = PSPL_model.PSPLmodel(multi_event)

# Fit with differential evolution
multi_fit = DE_fit.DEfit(multi_pspl, loss_function='chi2')
multi_fit.fit()

print("Multi-telescope fit results:")
print(multi_fit.fit_results['best_model'])
print(f"\nChi-squared: {multi_fit.fit_results['chi2']:.2f}")

# Plot results
pyLIMA_plots.plot_lightcurves(multi_pspl, multi_fit.fit_results['best_model'])
plt.title("Multi-Telescope pyLIMA Fit")
plt.show()

# Show flux parameters for each telescope
print("\nFlux parameters for each telescope:")
param_names = list(multi_pspl.model_dictionnary.keys())
for i, tel in enumerate(multi_event.telescopes):
    flux_param = multi_fit.fit_results['best_model'][3 + i]  # Flux parameters start at index 3
    print(f"{tel.name} ({tel.camera_filter}): {flux_param:.2f}")

### <font face="Helvetica" size="5"> 2.5 Performance and Best Practices </font>

pyLIMA offers several fitting algorithms, each with different strengths:

**Fitting Algorithms:**
- **LM (Levenberg-Marquardt)**: Fast, good for initial fits
- **DE (Differential Evolution)**: Robust, handles complex parameter spaces
- **MCMC**: Best for uncertainty estimation and posterior sampling
- **TRF (Trust Region Reflective)**: Good for constrained optimization

**Best Practices:**
- Start with LM for quick initial fits
- Use DE for complex models or when LM fails
- Use MCMC for final parameter estimation and uncertainties
- Set appropriate parameter bounds for better convergence
- Use custom parameters for better sampling (e.g., log_tE instead of tE)
- Always check event setup with `check_event()`

**Performance Tips:**
- Use `rescale_photometry=True` in MCMC for better numerical stability
- Set appropriate burn-in periods for MCMC chains
- Use simulation capabilities to test your analysis pipeline
- Consider using custom parameters for better sampling efficiency

## 3. RTModel

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;">

RTModel (Real-Time Model) is a sophisticated, hands-off microlensing modeling package developed by Valerio Bozza. Unlike other tools that require manual model selection, RTModel automatically determines the best model type for your event through a comprehensive grid search and template library approach. It's designed to be a "set it up and press go" solution that provides automated model interpretation.

**Key Features:**
- **Automated Model Selection**: Determines whether an event is single-lens, binary-lens, binary-source, etc.
- **Template Library**: Uses pre-computed binary lens templates for efficient fitting
- **Parallel Processing**: Exploits all available CPU cores for fast analysis
- **Comprehensive Models**: Supports 7+ model categories including parallax and orbital motion
- **Built-in Assessment**: Provides automatic interpretation of results
- **Visualization Tools**: Includes plotting and animation capabilities

### <font face="Helvetica" size="5"> 3.1 Installation and Setup </font>

RTModel requires a C++17 compiler and uses VBMicrolensing for calculations. It can be installed via pip or from source.

In [66]:
#@title Install RTModel

# Import RTModel
import RTModel
import RTModel.plotmodel as plm

print("RTModel installed and imported successfully!")
print(f"RTModel version: {RTModel.__version__}" if hasattr(RTModel, '__version__') else "RTModel imported")
wd = Path(os.getcwd())
print(f"Current working directory: {wd}")

RTModel installed and imported successfully!
RTModel version: 3.0
Current working directory: /Users/malpas.1/Code/data-challenge-notebooks/Extras


In [68]:
#@title Download RTModel example event (if needed)
# Assumes stdlib helpers + requests were imported near the top (see "General Imports").
os.chdir(wd)  # ensure we're in the original working directory
EVENT_DIR = Path("rtmodel_event001")
USE_DONE_ZIP = True  # set False to use raw event only

URL_RAW = "https://raw.githubusercontent.com/valboz/RTModel/refs/heads/main/events/event001.zip"
URL_DONE = "https://raw.githubusercontent.com/valboz/RTModel/refs/heads/main/events/event001done.zip"
url = URL_DONE if USE_DONE_ZIP else URL_RAW

if (EVENT_DIR / "Data").exists():
    print(f"{EVENT_DIR} already present; skipping download.")
else:
    print(f"Downloading RTModel example data from: {url}")
    with tempfile.TemporaryDirectory() as tmp:
        tmp_path = Path(tmp)
        zip_path = tmp_path / "event.zip"

        with requests.get(url, stream=True, timeout=60) as r:
            r.raise_for_status()
            with open(zip_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=1024 * 1024):
                    if chunk:
                        f.write(chunk)

        with zipfile.ZipFile(zip_path) as zf:
            names = [n for n in zf.namelist() if n and not n.endswith("/")]
            top_levels = sorted({n.split("/", 1)[0] for n in names if "/" in n})
            zf.extractall(tmp_path)

        if len(top_levels) == 1 and (tmp_path / top_levels[0]).exists():
            src = tmp_path / top_levels[0]
            if EVENT_DIR.exists():
                shutil.rmtree(EVENT_DIR)
            shutil.move(str(src), str(EVENT_DIR))
        else:
            EVENT_DIR.mkdir(parents=True, exist_ok=True)
            for item in list(tmp_path.iterdir()):
                if item.name == "event.zip":
                    continue
                if item.name == EVENT_DIR.name:
                    if EVENT_DIR.exists():
                        shutil.rmtree(EVENT_DIR)
                    shutil.move(str(item), str(EVENT_DIR))
                    break
                shutil.move(str(item), str(EVENT_DIR / item.name))

    print(f"Ready: {EVENT_DIR.resolve()}")

rtmodel_event001 already present; skipping download.


### 3.2 Data Preparation

RTModel requires a specific directory structure and data format. Each event needs its own directory with a `/Data` subdirectory containing the photometry files.

In [69]:
#@title Create RTModel Directory Structure
os.chdir(wd)  # ensure we're in the original working directory

# Create event directory structure for RTModel
data_dir = os.path.join(EVENT_DIR, 'Data')

# Create directories
os.makedirs(data_dir, exist_ok=True)

print(f"Created RTModel directory structure:")
print(f"  {EVENT_DIR}/")
print(f"  {EVENT_DIR}/Data/")

# RTModel requires this specific structure:
# event_dir/
# ├── Data/
# │   ├── telescope1.dat
# │   ├── telescope2.dat
# │   └── event001.coordinates
# └── (other files created by RTModel)

Created RTModel directory structure:
  rtmodel_event001/
  rtmodel_event001/Data/


In [62]:
#@title Create RTModel Data Files
os.chdir(wd.parent)  # ensure we're in the original working directory
# RTModel expects data in a specific format:
# # Mag err HJD-2450000
# 19.0232 0.012 8370.1223
# 19.0150 0.011 8370.2421
# ...

# If example data was downloaded above, it should already include an TelescopeA.dat (or similar) file.
# Only generate a synthetic one if it's missing.
data_file = os.path.join(data_dir, 'TelescopeA.dat')

# Show first few lines
with open(data_file, 'r') as f:
    lines = f.readlines()
    print("\nFirst 5 lines of data file:")
    for i, line in enumerate(lines[:5]):
        print(f"  {i+1}: {line.strip()}")


First 5 lines of data file:
  1: # Mag err HJD
  2: 
  3: 17.733916304426643 0.017916600891089943 8346.505460999906
  4: 
  5: 17.69898081443053 0.0179166008883137 8346.515986999962


In [70]:
#@title Create Coordinates File
os.chdir(wd)  # ensure we're in the original working directory
# RTModel requires event coordinates in a specific format
# Format: HH:MM:SS.S +DD:PP:SS.S

# If example data was downloaded above, it may already include event.coordinates.
coords_file = os.path.join(data_dir, 'event.coordinates')
if os.path.exists(coords_file):
    print(f"Found existing coordinates file: {coords_file} (skipping generation)")
else:
    # Convert our coordinates to the required format
    def decimal_to_hms(ra_deg, dec_deg):
        """Convert decimal degrees to HH:MM:SS.S +DD:PP:SS.S format"""
        # Convert RA (hours)
        ra_hours = ra_deg / 15.0  # Convert degrees to hours
        ra_h = int(ra_hours)
        ra_m = int((ra_hours - ra_h) * 60)
        ra_s = ((ra_hours - ra_h - ra_m/60) * 3600)

        # Convert Dec (degrees)
        dec_sign = '+' if dec_deg >= 0 else '-'
        dec_deg_abs = abs(dec_deg)
        dec_d = int(dec_deg_abs)
        dec_m = int((dec_deg_abs - dec_d) * 60)
        dec_s = ((dec_deg_abs - dec_d - dec_m/60) * 3600)

        ra_str = f"{ra_h:02d}:{ra_m:02d}:{ra_s:04.1f}"
        dec_str = f"{dec_sign}{dec_d:02d}:{dec_m:02d}:{dec_s:04.1f}"

        return f"{ra_str} {dec_str}"

    required = ['ra', 'dec']
    missing = [name for name in required if name not in globals()]
    if missing:
        raise NameError(
            "RTModel coordinates file is missing, and required variables were not found: "
            + ", ".join(missing)
            + ".\nRun the earlier cells that define ra/dec, or run the download cell above (USE_DONE_ZIP=True)."
        )

    coords_str = decimal_to_hms(ra, dec)
    with open(coords_file, 'w') as f:
        f.write(coords_str)

    print(f"Created coordinates file: {coords_file}")
    print(f"Coordinates: {coords_str}")
    print(f"Original coordinates: RA={ra:.6f}°, Dec={dec:.6f}°")

Found existing coordinates file: rtmodel_event001/Data/event.coordinates (skipping generation)


#### Satelite Observers

Satellites data are identified through their photometry filename, which **must** end with a number; e.g., `Spitzer1.dat`. Each satellite is identified by a unique number, which **must match** the number that ephemerides filename ends with; e.g., `satellite1.txt`. All other files ending by anything that is not a number are considered as taken from a ground telescope.

You can use the [JPL Horizons Tool](http://ssd.jpl.nasa.gov/horizons.cgi) to generate ephemerides files. The table settings should be:
1. Check Astrometric RA & Dec
2. Check Observer range & range-rate
3. Date/time format should be Julian Day calendar
4. Angle format should be decimal degrees
5. Range units should be astronomical units.

These tables can also be generated using astroquery and the horiszons API.

In [82]:
from astroquery.jplhorizons import Horizons
os.chdir(wd)  # ensure we're in the original working directory

def generate_ephemerides_file(filename, start_time, end_time, step, location, id='@0'):
    """Generate ephemerides file using astroquery and JPL Horizons API"""
    path = Path(filename)
    path.parent.mkdir(parents=True, exist_ok=True)  # <-- make sure directory exists

    obj = Horizons(id=id, location=location, epochs={'start': start_time, 'stop': end_time, 'step': step})
    eph = obj.ephemerides()

    with path.open('w') as f:
        f.write("# Ephemerides for RTModel\n")
        f.write("# Columns: JD RA(deg) Dec(deg) Range(AU) Range-rate(km/s)\n")
        for row in eph:
            jd = row['datetime_jd']
            ra_deg = row['RA']
            dec_deg = row['DEC']
            range_au = row['delta']
            range_rate_kms = row['delta_rate'] * 1731.456
            f.write(f"{jd:.5f} {ra_deg:.6f} {dec_deg:.6f} {range_au:.6f} {range_rate_kms:.3f}\n")


ephem_file = wd / "satelite1.txt"

if os.path.exists(ephem_file):
    print(f"Found existing ephemerides file: {ephem_file} (skipping generation)")
else:
    # generate and L2 ephemerides file for Jan 1, 2027 to Dec 31, 2027 with daily cadence
    start_time = '2027-01-01'
    end_time = '2027-12-31'
    location = '500'  # L2 point
    id = '@-170' # JPL Horizons ID for L2 point
    generate_ephemerides_file(ephem_file, 
        start_time=start_time, 
        end_time=end_time, 
        step='1d', 
        location=location, id=id)
    print(f"Generated ephemerides file: {ephem_file}")
    
# rename Data/TelescopeB.dat as Data/Telescope1.dat for mock parallax fitting
telescope_b_file = os.path.join(data_dir, 'TelescopeB.dat')
telescope_1_file = os.path.join(data_dir, 'Telescope1.dat')
if os.path.exists(telescope_1_file):
    print(f"Found existing {telescope_1_file} (skipping rename)")
elif os.path.exists(telescope_b_file):
    os.rename(telescope_b_file, telescope_1_file)
    print(f"Renamed {telescope_b_file} to {telescope_1_file} for mock parallax fitting")

Found existing ephemerides file: /Users/malpas.1/Code/data-challenge-notebooks/Extras/satelite1.txt (skipping generation)
Renamed rtmodel_event001/Data/TelescopeB.dat to rtmodel_event001/Data/Telescope1.dat for mock parallax fitting


### 3.3 Basic Usage

RTModel is designed to be simple to use - just set up your data and run the analysis. The tool automatically determines the best model type.

In [ ]:
#@title Initialize RTModel
os.chdir(wd)  # ensure we're in the original working directory

# Create RTModel instance
rtm = RTModel.RTModel(EVENT_DIR)

# Check number of processors (RTModel uses all available by default)
print(f"RTModel initialized for event: {EVENT_DIR}")
print(f"Available processors: {rtm.nprocessors}")

# You can limit the number of processors if needed
# rtm.set_processors(4)  # Use only 4 processors

*********************
****   RTModel   ****
*********************
Event name: /Users/malpas.1/Code/data-challenge-notebooks/Extras/rtmodel_event001
Number of processors: 8
RTModel initialized for event: rtmodel_event001
Available processors: 8


RTModel will automatically:
  1. Pre-process the data
  2. Generate initial conditions
  3. Fit all model categories
  4. Select best models
  5. Provide final assessment

In [ ]:
#@title Run RTModel Analysis
os.chdir(wd)  # ensure we're in the original working directory

# This is the main command - RTModel does everything automatically!
print("Starting RTModel analysis...")
print("This may take several minutes depending on your machine.")
print("RTModel will fit 7 different model categories:")
print("  PS: Single-lens-single-source")
print("  PX: Single-lens-single-source with parallax")
print("  BS: Single-lens-binary-source")
print("  BO: Single-lens-binary-source with xallarap")
print("  LS: Binary-lens-single-source")
print("  LX: Binary-lens-single-source with parallax")
print("  LO: Binary-lens-single-source with orbital motion")

# Set RUN to True to actually run RTModel
RUN = False
if RUN:
    rtm.run()
else:
    print("\nNote: RTModel run skipped out to avoid long execution.")
    print("Set RUN to True to perform the actual analysis.")

Starting RTModel analysis...
This may take several minutes depending on your machine.
RTModel will fit 7 different model categories:
  PS: Single-lens-single-source
  PX: Single-lens-single-source with parallax
  BS: Single-lens-binary-source
  BO: Single-lens-binary-source with xallarap
  LS: Binary-lens-single-source
  LX: Binary-lens-single-source with parallax
  LO: Binary-lens-single-source with orbital motion
o Mon Feb 16 23:16:17 2026
- Analyzing event:  /Users/malpas.1/Code/data-challenge-notebooks/Extras/rtmodel_event001
- Launching: Reader
  Pre-processing data...
  OK
o Mon Feb 16 23:16:17 2026
- Launching: InitCond
  Setting initial conditions...
Peaks:  8550.1520  8555.3180  
  OK
o Mon Feb 16 23:16:18 2026
- Single-lens-Single-source fits
Fits completed: 100%|██████████| 120/120 [00:03<00:00, 38.71it/s]
  OK
o Mon Feb 16 23:16:21 2026
- Selecting models for Single-lens-Single-source fits
  OK
o Mon Feb 16 23:16:21 2026
- Single-lens-Single-source fits with parallax
Fits c

### 3.4 Understanding RTModel Output

After running RTModel, you'll get several output files and directories. Let's explore what each contains.

**RTModel creates the following output structure:**

```
<event_dir>/
├── Data/                    # Original input data
├── ini/                     # Configuration files
├── InitCond/                # Initial conditions for fitting
├── Models/                  # Selected models for each category
├── FinalModels/             # Best models from final assessment
├── LCToFit.txt              # Pre-processed data
├── FilterToData.txt         # Dataset mapping
├── spline.txt               # Spline approximation points
└── nature.txt               # Final assessment and best models
```

**Key output files:**
  `nature.txt`: Contains the final assessment and list of best models
  `FinalModels/`: Contains the best model files with parameters and uncertainties
  `Models/`: Contains selected models for each category

#### Understanding the nature.txt File

The `nature.txt` file contains RTModel's assessment of the event

**The nature.txt file contains:**

1. Best chi-square for each model category:
  - `PS`: Single-lens-single-source
  - `PX`: Single-lens-single-source with parallax
  - `BS`: Single-lens-binary-source
  - `BO`: Single-lens-binary-source with xallarap
  - `LS`: Binary-lens-single-source
  - `LX`: Binary-lens-single-source with parallax
  - `LO`: Binary-lens-single-source with orbital motion

2. Final assessment of the event nature

3. List of proposed best models

4. Model comparison and interpretation

**Example `nature.txt` content:**

```nature.txt
*********************
****   RTModel   ****
*********************

Best chi-square for each category:
PS: 1234.56 (Single-lens-single-source)
PX: 1230.45 (Single-lens-single-source with parallax)
BS: 1235.67 (Single-lens-binary-source)
BO: 1232.34 (Single-lens-binary-source with xallarap)
LS: 1238.90 (Binary-lens-single-source)
LX: 1235.12 (Binary-lens-single-source with parallax)
LO: 1236.78 (Binary-lens-single-source with orbital motion)

Final Assessment:
This appears to be a single-lens event with parallax effects.
The best model is PX with chi-square = 1230.45
```

### 3.5 Model Categories

RTModel fits 7 different model categories by default. Each has a specific label and parameter set.

#### RTModel Model Categories

| Label | Model | Parameters | Description |
|-------|-------|------------|-------------|
| `PS` | Single-lens-single-source | 4 | Basic microlensing (`t0`, `u0`, `tE`, `rho`) |
| `PX` | Single-lens-single-source with parallax | 6 | Basic + parallax effects |
| `BS` | Single-lens-binary-source | 7 | Two source stars, one lens |
| `BO` | Single-lens-binary-source with xallarap | 10 | Binary source + orbital motion |
| `LS` | Binary-lens-single-source | 7 | Two lens components (planetary) |
| `LX` | Binary-lens-single-source with parallax | 9 | Binary lens + parallax |
| `LO` | Binary-lens-single-source with orbital motion | 12 | Binary lens + orbital motion |

#### Additional categories available:
- `LK`: Binary-lens-single-source with eccentric orbital motion (14 parameters)

**Parameter details for each model category:**

| Parameter | PS | PX | BS | BO | LS | LX | LO | Notes |
|-----------|----|----|----|----|----|----|----|-------|
| `u0` | ✅* | ✅ | ✅ | ✅ | ✅ | ✅ | ✅ | Impact parameter |
| `tE` | ✅* | ✅* | ✅* | ✅* | ✅* | ✅* | ✅* | Einstein time in days |
| `t0` | ✅ | ✅ | ✅ | ✅ | ✅ | ✅ | ✅ | Closest approach time in HJD |
| `rho` | ✅* | ✅* | ✅* | ✅* | ✅* | ✅* | ✅* | Source radius |
| `piN` | ❌ | ✅ | ❌ | ❌ | ❌ | ✅ | ❌ | Parallax component along North |
| `piE` | ❌ | ✅ | ❌ | ❌ | ❌ | ✅ | ❌ | Parallax component along East |
| `s` | ❌ | ❌ | ❌ | ❌ | ✅* | ✅* | ✅* | Separation between lenses |
| `q` | ❌ | ❌ | ❌ | ❌ | ✅* | ✅* | ✅* | Mass ratio |
| `alpha` | ❌ | ❌ | ❌ | ❌ | ✅ | ✅ | ✅ | Angle of source trajectory |
| `q_S` | ❌ | ❌ | ✅ | ✅ | ❌ | ❌ | ❌ | Source flux ratio |
| `rho_S` | ❌ | ❌ | ✅* | ✅* | ❌ | ❌ | ❌ | Source separation |
| `theta_S` | ❌ | ❌ | ✅ | ✅ | ❌ | ❌ | ❌ | Source angle |

> key: `*` - ln scale

### <font face="Helvetica" size="5"> 3.6 Visualization and Results </font>

RTModel includes built-in visualization tools through the `plotmodel` subpackage.

1. Light curve plotting:

    ```python
    plm.plotmodel(eventname, modelfile) # Plot light curve with model
    plm.plotmodel(eventname, modelfile, show_caustics=True) # Show caustics
    ```
2. Animation capabilities:

    ```python
    plm.animate_model(eventname, modelfile) # Create animated GIF
    ```
3. Astrometric plots:

    ```python
    plm.plot_astrometry(eventname, modelfile) # Plot astrometric trajectory
    ```

In [ ]:
#@title RTModel Plotting
os.chdir(wd)  # ensure we're in the original working directory

# Example usage:

# After running RTModel and getting results, check the model 
# file names in the 'FinalModels' directory created by RTModel, 
# and update the modelfile argument accordingly.

if RUN:
    print("\nPlotting RTModel results...")
    plm.plotmodel(
        eventname='rtmodel_event001', 
        modelfile='FinalModels/LX0001-1.txt'
        )
    plm.plotmodel(
        eventname='rtmodel_event001', 
        modelfile='FinalModels/LX0001-2.txt', 
        show_caustics=True
        )

#### How to analyze RTModel results:

1. Check nature.txt for the final assessment

   - Best chi-square for each model category
   - RTModel's interpretation of the event
   - List of proposed best models

2. Examine FinalModels/ directory

   - Each file contains parameters and uncertainties
   - First line: parameters + fluxes + chi-square
   - Second line: parameter uncertainties
   - Remaining lines: covariance matrix

3. Compare models using chi-square

   - Lower chi-square = better fit
   - Consider degrees of freedom (more parameters = higher expected chi-square)
   - Use F-test or AIC/BIC for model comparison

4. Validate results

   - Check parameter uncertainties
   - Examine residuals
   - Look for systematic effects
   - Consider physical plausibility

### 3.7 Advanced Features

RTModel offers several advanced features for customization and control.

In [ ]:
#@title: 1. Model Category Selection:

# Fit only specific model categories
rtm.set_model_categories(['PS', 'LS'])  # Only single-lens and binary-lens

In [ ]:
#@title: 2. Fitting Options:

# Configure Levenberg-Marquardt fitting
rtm.config_LevMar(nfits=10, timelimit=1200.0, maxsteps=100)

In [ ]:
#@title: 3. Parameter Constraints:

# Set gaussian constraints on parameters
rtm.set_constraints('tE', 25.0, 2.0)  # tE = 25 ± 2 days

In [ ]:
#@title: 4. Parallel Processing:

# Control number of processors
rtm.set_processors(8)  # Use 8 processors

In [ ]:
#@title: 5. Step-by-step Execution:

# Run individual steps manually
rtm.Reader()           # Data pre-processing
rtm.InitCond()         # Generate initial conditions
rtm.launch_fits('PS')  # Fit specific category
rtm.ModelSelection()   # Select best models
rtm.FinalAssessment()  # Final assessment

### 3.8 RTModel Best Practices

**1. Data Quality:**
- Ensure data is properly formatted
- Check for systematic errors
- Verify coordinate accuracy

**2. Computational Resources:**
- Use all available processors for speed
- Allow sufficient time (1-3 hours typical)
- Ensure adequate disk space

**3. Model Interpretation:**
- Don't rely solely on chi-square
- Consider physical plausibility
- Check parameter uncertainties
- Validate against known physics

**4. Troubleshooting:**
- Check ini/ directory for configuration
- Examine error messages in output
- Verify data format compliance
- Use step-by-step execution for debugging

**5. Results Validation:**
- Compare with other fitting codes
- Check for systematic residuals
- Verify parameter correlations
- Consider alternative models

### 3.9 RTModel vs Other Tools

RTModel offers a unique approach compared to other microlensing fitting tools.

#### <font face="Helvetica" size="4"> Comparison with Other Tools </font>

**RTModel vs Other Microlensing Tools:**

| Feature | RTModel | MulensModel | pyLIMA |
|---------|---------|-------------|--------|
| Model Selection | Automatic | Manual | Manual |
| Ease of Use | Moderate set up <br> Very easy to run | Moderate | Moderate |
| Speed | Fast (parallel) | Variable | Variable |
| Automation | High | Low | Low |
| Customization | Moderate | High | High |
| Visualization | Built-in | Basic | Good |
| Parallel Processing | Yes | Manual | Manual |

**RTModel Advantages:**
- Hands-off operation - just set up data and run
- Automatic model selection and interpretation
- Template library for efficient binary lens fitting
- Built-in assessment and visualization
- Parallel processing for speed

**RTModel Limitations:**
- Less customization than manual tools
- Requires specific data format
- Limited to supported model categories
- May miss subtle effects in complex events

### 3.10 Astrophotometric Fitting (RTModel v3.0)

**NEW IN RTModel v3.0!** RTModel now supports combined photometric and astrometric fitting, making it the first tool to offer comprehensive astrophotometric microlensing analysis. This capability is crucial for space-based observations like Roman, where milliarcsecond astrometric precision enables measurement of the centroid trajectory during microlensing events.

**Key Astrometric Features:**
- **Combined Fitting**: Simultaneous photometric and astrometric parameter estimation
- **Centroid Trajectory**: Models the astrometric shift of images during events
- **Proper Motion**: Measures source and lens proper motions
- **Einstein Angle**: Direct measurement of θ_E from astrometry
- **Source Parallax**: Geometric parallax of the source star
- **Visualization**: Built-in astrometric plotting and trajectory visualization

#### 3.10.1 Astrophotometric Data Format

Astrophotometric datasets include both photometric and astrometric information in a single file.

Astrophotometric datasets have 7 columns: `Mag`, `err`, `HJD-2450000`, `Dec`, `errDec`, `RA`, `errRA`

**Example astrophotometric data:**

```
# Mag err HJD-2450000 Dec errDec RA errRA
19.0232 0.012 8370.1223 1.2534 1.0 0.0165 1.0
19.0150 0.011 8370.2421 1.7510 1.1 -0.5422 1.1
19.0034 0.011 8370.3697 1.1190 1.1 -0.1981 1.1
18.9712 0.010 8370.4911 1.4281 1.0 0.2119 1.0
18.9592 0.011 8370.6114 1.3005 0.9 0.3982 1.0
18.9109 0.009 8370.8234 1.6233 1.0 0.5121 1.0
18.8798 0.009 8371.0092 2.0223 1.2 0.9411 1.1
```

**Column descriptions:**

  - `Mag`: Magnitude
  - `err`: Photometric error
  - `HJD-2450000`: Heliocentric Julian Date - 2450000
  - `Dec`: Declination offset in milliarcseconds
  - `errDec`: Declination error in milliarcseconds
  - `RA`: Right Ascension offset in milliarcseconds
  - `errRA`: Right Ascension error in milliarcseconds

> Note: Dec and RA are angular displacements from a fixed reference point
in the North and East directions respectively.

In [ ]:
#@title Create Astrophotometric Data
# Create sample astrophotometric data
def create_astrophotometric_data(filename, times, mags, mag_errs):
    """Create RTModel-compatible astrophotometric data file"""
    with open(filename, 'w') as f:
        f.write("# Mag err HJD-2450000 Dec errDec RA errRA\n")
        for t, m, e in zip(times, mags, mag_errs):
            # Convert to HJD-2450000 format
            hjd_offset = t - 2450000

            # Generate synthetic astrometric data
            # Simulate centroid motion during microlensing event
            t0 = 2459123.5  # Peak time
            tE = 25.0       # Einstein time

            # Simple astrometric model (centroid shift)
            u = abs(t - t0) / tE
            if u < 0.1:  # Near peak
                dec_offset = 2.0 * np.exp(-u**2)  # Peak shift
                ra_offset = 1.5 * np.exp(-u**2)   # Peak shift
            else:
                dec_offset = 0.5 * np.exp(-u**2)  # Background
                ra_offset = 0.3 * np.exp(-u**2)   # Background

            # Add noise
            dec_offset += np.random.normal(0, 0.1)
            ra_offset += np.random.normal(0, 0.1)

            # Astrometric errors (typically 0.5-1.0 mas)
            dec_err = 0.8 + 0.2 * np.random.random()
            ra_err = 0.8 + 0.2 * np.random.random()

            f.write(f"{m:.4f} {e:.3f} {hjd_offset:.4f} {dec_offset:.4f} {dec_err:.1f} {ra_offset:.4f} {ra_err:.1f}\n")

# Create astrophotometric data file
astro_data_file = os.path.join(data_dir, 'roman_astrometric.dat')
create_astrophotometric_data(astro_data_file, times, mag, mag_err)

print(f"Created astrophotometric data file: {astro_data_file}")
print(f"Data points: {len(times)}")

# Show first few lines
with open(astro_data_file, 'r') as f:
    lines = f.readlines()
    print("\nFirst 5 lines of astrophotometric data:")
    for i, line in enumerate(lines[:5]):
        print(f"  {i+1}: {line.strip()}")

#### 3.10.2 Astrophotometric Parameters

When RTModel detects astrophotometric data, it automatically adds 4 additional parameters to the fit:

| Parameter | Meaning | Physical Significance |
|-----------|---------|---------------------|
| `muS_Dec` | Source proper motion component in North direction (mas/yr) | Source motion perpendicular to line of sight |
| `muS_RA` | Source proper motion component in East direction (mas/yr) | Source motion perpendicular to line of sight |
| `piS` | Geometric parallax of the source (mas) | Distance to source star |
| `thetaE` | Einstein angle (mas) | Lens mass and distance (θ_E = √(4GM/c²D_L)) |

**Model Categories for Astrophotometric Fits:**
* `PX`: Single-lens-single-source with parallax (10 parameters)
* `BO`: Single-lens-binary-source with xallarap (14 parameters)
* `LX`: Binary-lens-single-source with parallax (13 parameters)
* `LO`: Binary-lens-single-source with orbital motion (16 parameters)
* `LK`: Binary-lens-single-source with eccentric orbital motion (18 parameters)

> Note: No static models are fitted in astrophotometric mode because parallax is always included to model the centroid trajectory.

#### 3.10.3 Astrophotometric Fitting

RTModel automatically detects astrophotometric datasets and switches to combined fitting mode.

**RTModel Astrophotometric Fitting:**

1. Automatic Detection:

  RTModel automatically detects astrophotometric datasets by checking for 7-column format (vs 3-column for photometric)

2. Combined Chi-Square:

  * $\chi^2_{total} = \chi^2_{photometric} + \chi^2_{astrometric}$
  * Both contributions are weighted appropriately

3. Enhanced Parameter Space:

   - 4 additional astrometric parameters
   - Parallax always included (no static models)
   - More complex model selection

4. Example Usage:

  ```python
  # RTModel automatically detects astrophotometric data
  rtm = RTModel.RTModel('astro_event')
  rtm.run()  # Automatically uses astrophotometric fitting
  ```

5. Output Structure:

  * Model files contain: `nps + 4*ntel + 1` parameters
   - `nps`: model parameters + 4 astrometric parameters
   - `4*ntel`: fluxes and centroid positions for each telescope
   - `+1`: total chi-square

#### 3.10.4 Astrometric Visualization

RTModel includes specialized plotting functions for astrometric data visualization.

**RTModel Astrometric Plotting Functions:**
   

In [ ]:
#@title: 1. Combined Light Curve and Astrometry
# Shows both photometric light curve and astrometric parameters

if RUN:
    myplot = plm.plotmodel(
        'rtmodel_event001', 
        'LX0001',
        modelfile='FinalModels/LX0001-1.txt'
    )
    myplot.show_lightcurve()

In [ ]:
#@title: 2. Centroid Trajectory in the Sky:")
# Shows the centroid trajectory as a function of time
# Displays RA vs Dec with error ellipses

if RUN:
    myplot.showastrometry()

In [ ]:
#@title: 3. Individual Astrometric Components:")

if RUN:
    myplot.showastrometryRA()  # RA vs time
    myplot.showastrometryDec() # Dec vs time

In [ ]:
#@title: 4. Multiple Telescope Support:")
# Different telescopes may have different blending fractions

if RUN:
    myplot.showastrometry(1)  # Show astrometry for telescope 1

Key Features:

   - Automatic error ellipse plotting
   - Time evolution of centroid position
   - Comparison with observed astrometric data
   - Proper motion vector visualization

#### 3.10.5 Physical Interpretation of Astrophotometric Parameters

1. Einstein Angle ($\theta_E$)
> $\theta_E = \sqrt{(4GM/c^2D_L)}$
- Direct measurement of lens mass and distance
- Breaks the mass-distance degeneracy
- Typical values: 0.1-1.0 mas for stellar lenses

2. Source Proper Motion ($\mu_S$)
> $\mu_S = v_S / D_S$
- Tangential velocity of source star
- Helps constrain source distance and kinematics
- Typical values: 1-10 mas/yr for Galactic sources

3. Source Parallax ($\pi_S$)
$\pi_S = 1/D_S$
- Geometric parallax of source star
- Provides independent distance measurement
- Typical values: $0.1-1.0 \,\text{mas}$ for Galactic sources

4. Centroid Trajectory
- Shows the motion of the light centroid during the event
- Amplitude depends on lens mass and source-lens separation
- Shape reveals lens geometry (single vs binary)
- Duration related to Einstein time

5. Advantages of Astrophotometric Fitting
- Breaks degeneracies in lens mass and distance
- Provides independent constraints on source properties
- Enables direct measurement of Einstein angle
- Improves parameter precision and accuracy
- Essential for space-based microlensing surveys

## 4. popclass: Probabilistic Classification of Microlensing Lenses

### What is popclass?

**popclass** is a Python package developed by LLNL (lead: Peter McGill) for *probabilistic classification* of the lens in a gravitational microlensing event. It bridges the gap between event posteriors (from light curve modeling) and population synthesis simulations of the Milky Way, allowing you to infer the probability that a given event was caused by a star, white dwarf, neutron star, or black hole.

- **Key use:** Given posterior samples (e.g., from MulensModel, pyLIMA, or any Bayesian fit) and a Galactic population model, popclass computes the probability that the lens belongs to each class.
- **Why use it?** It enables population-level inference, e.g., identifying likely black hole events, and is designed for the era of large surveys (Roman, Rubin).

### <font face="Helvetica" size="5"> 4.1 Installation </font>

```bash
pip install popclass
```
or via conda:
```bash
conda install -c conda-forge popclass
```

### <font face="Helvetica" size="5"> 4.2 Basic Usage Example </font>

Suppose you have posterior samples for an event in log10(tE) and log10(piE):

In [ ]:
from popclass.posterior import Posterior
from popclass.model import PopulationModel
from popclass.classify import classify
import popclass.model as pm

#copy_arrays is not supported in our version of asdf (5.1.0) but popclass 
# expects it, so we monkey-patch it here to avoid errors.
_asdf_open = pm.asdf.open
def _open_compat(*args, **kwargs):
    kwargs.pop("copy_arrays", None)
    return _asdf_open(*args, **kwargs)

pm.asdf.open = _open_compat

# Mock posterior samples
NUM_POSTERIOR_SAMPLES = 10000
logtE = np.random.normal(loc=2, scale=0.1, size=NUM_POSTERIOR_SAMPLES)
logpiE = np.random.normal(loc=-1, scale=0.5, size=NUM_POSTERIOR_SAMPLES)
posterior_samples = np.vstack((logtE, logpiE)).T
prior_density = 0.028 * np.ones(NUM_POSTERIOR_SAMPLES)  # uniform prior

# Wrap in popclass objects
posterior = Posterior(samples=posterior_samples, parameter_labels=['log10tE', 'log10piE'])
inference_data = posterior.to_inference_data(prior_density)

# Load a pre-built population model (e.g., PopSyCLE)
popsycle = PopulationModel.from_library('popsycle_singles_sukhboldn20')

# Classify!
classification = classify(population_model=popsycle, inference_data=inference_data, parameters=['log10tE', 'log10piE'])
print(classification)

{'black_hole': 0.09671087911996425, 'neutron_star': 0.0014161261827535846, 'star': 0.7049543986944241, 'white_dwarf': 0.19691859600285797}


You should get something like:

**Output:**
`{'black_hole': 0.09, 'neutron_star': 0.001, 'star': 0.71, 'white_dwarf': 0.20}`

These are the relative probabilities of our made up lens belonging to each of the classified groups.

If you have run an MCMC fit with parallax before this step in the notebook, consider running this classifier on your posterior sample as an exercise. Visit the [`popclass` documentation site](https://popclass.readthedocs.io/en/latest/), if you get stuck.


### 4.3 How It Works

- **Inputs:** Posterior samples for event parameters (e.g., `tE`, `piE`), their prior densities, and a Galactic population model (e.g., from PopSyCLE).
- **Method:** Uses Bayesian importance sampling to compare the event's posterior to simulated populations, computing the probability for each lens class.
- **Output:** A dictionary of class probabilities (star, white dwarf, neutron star, black hole).

**Mathematical summary:**

$$
p(\text{class}_L| \boldsymbol{d}, \mathcal{G}) = \frac{p(\text{class}_L| \mathcal{G})}{p(\boldsymbol{d}| \mathcal{G})}
    \times \frac{1}{S} \sum _{c=0}^{S} \frac{p(\theta _c | \text{class}_L, \mathcal{G})}{\pi(\theta _{c})}
$$

where:
- $ \boldsymbol{d} $: event data
- $ \mathcal{G} $: Galactic model
- $ \theta_c $: posterior samples
- $ \pi(\theta_c) $: prior density

### 4.4 Population Models

popclass comes with several pre-built population models (from PopSyCLE), e.g.:
- `popsycle_singles_sukhboldn20`
- `popsycle_singles_raithel18`
- `popsycle_singles_spera15`

You can also load your own models in ASDF format.

### 4.5 Advanced Features

- **Flexible parameter spaces:** Use any set of event parameters (e.g., `tE`, `piE`, `thetaE`, `blend_fraction`).
- **Custom population models:** Supply your own simulation data in the required format.
- **Uncertainty quantification:** Includes a "None" class to flag events not well explained by the model.
- **ArviZ and PyMultiNest integration:** For handling posteriors from common Bayesian tools.
- **Plotting:** Built-in tools for visualizing classification results.

### 4.6 Best Practices & Caveats

- **Parameter consistency:** Ensure your posterior and population model use the same parameterization (e.g., both in `log10(tE)`, not `tE`).
- **Prior density:** Must match the parameter space of the population model (apply Jacobian if transforming variables).
- **Model completeness:** If your event is outside the simulated parameter space, the "None" class will help flag this.
- **Interpretation:** Probabilities are only as good as the population model and the event posterior.

### 4.7 Further Reading 

- [popclass documentation](https://popclass.readthedocs.io)
- [PopSyCLE population models](https://github.com/jluastro/PopSyCLE)
- Perkins et al. (2024), Kaczmarek et al. (2024) — theoretical background

> **popclass** is a powerful tool for population-level inference in microlensing, enabling robust, probabilistic classification of lens types for large event samples.

## Contributing

We welcome contributions to improve this tutorial! Whether you want to add new tools, enhance existing sections, or fix issues, please check out our contributing guidelines below.

<div align="center">
  <a href="https://github.com/rges-pit/data-challenge-notebooks/blob/main/CONTRIBUTING.md">
    <img src="https://github.com/AmberLee2427/microlens-submit/blob/main/docs/_static/github-desktop_logo.png?raw=1" alt="GitHub" width="20" height="20" style="vertical-align: middle; margin-right: 8px;"/>
    <span style="font-size: 16px; font-weight: 500; color: #0366d6; text-decoration: none;">Contributing to This Tutorial</span>
  </a>
</div>

## About this Notebook

**Author(s):** Amber Malpas, Ali Crisp
**Maintainers:** RGES-PIT Working Group 9  
**Last Updated:** 20 Nov 2025  
**Contact:** malpas.1@osu.edu

## Citations

If you use `MulensModel`, `pyLIMA`, `RTModel`, `popClass`, or this notebook for published research, please cite the
authors. Follow these links for more information about citing:

* [Citing `MulensModel`](https://github.com/rpoleski/MulensModel/blob/master/CITATION.cff)
* [Citing `pyLIMA`](https://github.com/ebachelet/pyLIMA?tab=readme-ov-file#citations)
* [Citing `RTModel`](https://github.com/valboz/RTModel#attribution)
* [Citing `popClass`](https://github.com/LLNL/popclass/blob/main/CITATION.cff)
* [Citing **Roman Microlensing Data Challenge 2026 Notebooks**](https://github.com/rges-pit/data-challenge-notebooks/blob/main/zenodo.txt)

### Citing Roman Microlensing Data Challenge 2026 Notebooks

If you use our notebooks in your project, please cite:

```
Malpas, A., Murlidhar, A., Vandorou, K., Kruszyńska, K., Crisp, A., & Vyas, M. (2026). Roman Microlensing Data Challenge 2026 Notebooks (v1.0.0). Zenodo. https://doi.org/10.5281/zenodo.18262183
```

**BibTeX:**
```bibtex
@software{malpas_2025_17806271,
  author       = {Malpas, Amber and Murlidhar, Arjun and Vyas, Meet and Vandorou, Katie and Kruszyńska, Katarzyna and Crisp, Ali},
  title        = {Roman Microlensing Data Challenge 2026 Notebooks},
  month        = dec,
  year         = 2026,
  publisher    = {Zenodo},
  version      = {v1.0.0},
  doi          = {10.5281/zenodo.18262183},
  url          = {https://doi.org/10.5281/zenodo.18262183}
}
```

<!-- Footer Start -->

<!-- Footer End -->